<a href="https://colab.research.google.com/github/AmirulMukminin0/DSS-Thesis/blob/main/Analysis_rice_stress_detection_rgb_thermal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TAHAP 0 - Mount Google Drive & Set ROOT

In [ ]:
# ============================================
# TAHAP 0.1 — Mount Google Drive & Set ROOT
# ============================================
from google.colab import drive
drive.mount("/content/drive")

import os
from pathlib import Path

# === ROOT UTAMA (KUNCI) ===
PROJECT_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal")
FIX_ROOT     = PROJECT_ROOT / "DatasetTerbaruFIX"

# === SUBFOLDER FIX (biar tidak nyampur dengan lama) ===
FRAMES_DIR       = FIX_ROOT / "frames"
RGB_ALIGNED_DIR  = FRAMES_DIR / "rgb_aligned"
THERMAL_DIR      = FRAMES_DIR / "thermal"
PAIRS_CSV        = FRAMES_DIR / "pairs_index.csv"

PATCH_ROOT       = FIX_ROOT / "patches"
PATCH_RGB_DIR    = PATCH_ROOT / "rgb"        # optional if you choose to save patch images
PATCH_TH_DIR     = PATCH_ROOT / "thermal"    # optional if you choose to save patch npy
PATCH_META_DIR   = PATCH_ROOT / "meta"
PATCHES_ALL_CSV  = PATCH_META_DIR / "patches_all.csv"
PATCH_GRID_JSON  = PATCH_META_DIR / "patch_grid.json"

LABEL_ROOT       = FIX_ROOT / "labels"
LABEL_GOLD_DIR   = LABEL_ROOT / "gold"
LABEL_SILVER_DIR = LABEL_ROOT / "silver"
LABEL_FINAL_DIR  = LABEL_ROOT / "final"

THERM_PREP_DIR   = FIX_ROOT / "thermal_prep"
THERM_MASK_DIR   = THERM_PREP_DIR / "masks_v2"
THERM_STATS_DIR  = THERM_PREP_DIR / "stats"

EXP_ROOT         = FIX_ROOT / "experiments"
DEPLOY_ROOT      = FIX_ROOT / "deploy"
DEPLOY_APP_DIR   = DEPLOY_ROOT / "flask_app"
DEPLOY_ART_DIR   = DEPLOY_ROOT / "artifacts"

OUT_DIR          = FIX_ROOT / "outputs"

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

# Create key directories
for d in [
    FIX_ROOT,
    FRAMES_DIR, RGB_ALIGNED_DIR, THERMAL_DIR,
    PATCH_ROOT, PATCH_RGB_DIR, PATCH_TH_DIR, PATCH_META_DIR,
    LABEL_ROOT, LABEL_GOLD_DIR, LABEL_SILVER_DIR, LABEL_FINAL_DIR,
    THERM_PREP_DIR, THERM_MASK_DIR, THERM_STATS_DIR,
    EXP_ROOT, DEPLOY_ROOT, DEPLOY_APP_DIR, DEPLOY_ART_DIR,
    OUT_DIR
]:
    ensure_dir(d)

print("✅ Mounted Drive & Initialized Paths")
print("PROJECT_ROOT:", PROJECT_ROOT)
print("FIX_ROOT    :", FIX_ROOT)
print("FRAMES_DIR  :", FRAMES_DIR)
print("PATCH_ROOT  :", PATCH_ROOT)
print("LABEL_ROOT  :", LABEL_ROOT)
print("EXP_ROOT    :", EXP_ROOT)
print("DEPLOY_ROOT :", DEPLOY_ROOT)
print("OUT_DIR     :", OUT_DIR)

# sanity checks
print("\n--- Sanity Checks ---")
print("PROJECT_ROOT exists?", PROJECT_ROOT.exists())
print("FIX_ROOT exists?    ", FIX_ROOT.exists())


In [ ]:
# ============================================
# CEK JUMLAH PATCH: TOTAL, BERLABEL, TIDAK DIPAKAI
# ============================================

import pandas as pd
from pathlib import Path

from google.colab import drive
drive.mount("/content/drive")

# ROOT SESUAI NOTEBOOK ANDA
ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")

# ---- FILE PATCH HASIL PREPROCESSING ----
PATCH_ALL = ROOT / "patches" / "meta" / "patches_all.csv"

# ---- CARI FILE LABEL FINAL (AUTO) ----
LABEL_DIR = ROOT / "labels" / "final"
label_files = sorted(LABEL_DIR.glob("patches_all_final_labeled*.csv"))

assert PATCH_ALL.exists(), f"Tidak ditemukan: {PATCH_ALL}"
assert len(label_files) > 0, "File label final tidak ditemukan di labels/final/"

PATCH_LABELED = label_files[-1]  # ambil yang terbaru

# ---- LOAD DATA ----
df_all = pd.read_csv(PATCH_ALL)
df_labeled = pd.read_csv(PATCH_LABELED)

# ---- HITUNG ----
total_patch_after_preprocessing = len(df_all)
total_patch_labeled = len(df_labeled)
total_patch_unused = total_patch_after_preprocessing - total_patch_labeled

# ---- OUTPUT RINGKAS ----
summary = pd.DataFrame({
    "Jumlah Patch": [
        total_patch_after_preprocessing,
        total_patch_labeled,
        total_patch_unused
    ]
}, index=[
    "Total patch setelah preprocessing",
    "Total patch berlabel",
    "Total patch tidak dipakai"
])

print("📁 File preprocessing :", PATCH_ALL)
print("📁 File label final   :", PATCH_LABELED)
print("\n📊 Ringkasan Patch:")
summary


In [ ]:
# ============================================
# TAHAP 0.2 — Imports, Seed, Global Config
# ============================================
import random
import json
import time
import numpy as np
import pandas as pd

import cv2
from PIL import Image

# ML stack (TF/Keras assumed)
import tensorflow as tf

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Global config (KUNCI)
PATCH_SIZE = 128
STRIDE     = 128
ZOOM_RGB_TO_THERMAL = 2.2  # default zoom

# Thermal normalization config (will be used in Tahap 3.3)
THERM_P01 = 1.0
THERM_P99 = 99.0

# Canopy mask QC
MIN_CANOPY_FRAC = 0.20

# Runtime check
print("✅ Imports OK")
print("TensorFlow version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)

print("\n--- Global Config ---")
print("SEED =", SEED)
print("PATCH_SIZE =", PATCH_SIZE, "| STRIDE =", STRIDE)
print("ZOOM_RGB_TO_THERMAL =", ZOOM_RGB_TO_THERMAL)
print("THERM percentiles =", (THERM_P01, THERM_P99))
print("MIN_CANOPY_FRAC =", MIN_CANOPY_FRAC)


In [ ]:
# ============================================
# TAHAP 0.3 — Helper Functions (Core Utilities)
#   - load RGB / thermal
#   - center-crop + zoom + resize (RGB -> thermal grid)
#   - patch grid + extraction (thermal-domain)
# ============================================

def read_rgb(path: str | Path) -> np.ndarray:
    """Read RGB image (png/jpg) as uint8 RGB."""
    img_bgr = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if img_bgr is None:
        raise FileNotFoundError(f"RGB not readable: {path}")
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb

def read_thermal_npy(path: str | Path) -> np.ndarray:
    """Read thermal .npy float32 matrix (H,W)."""
    arr = np.load(str(path))
    if arr.ndim != 2:
        raise ValueError(f"Thermal must be 2D (H,W), got shape {arr.shape} from {path}")
    return arr.astype(np.float32)

def center_crop_zoom(img: np.ndarray, zoom: float) -> np.ndarray:
    """
    Center crop with zoom factor > 1.
    zoom=2 means crop to 1/2 width & height (then resize back later).
    """
    if zoom <= 1.0:
        return img
    h, w = img.shape[:2]
    new_w = int(round(w / zoom))
    new_h = int(round(h / zoom))
    x0 = (w - new_w) // 2
    y0 = (h - new_h) // 2
    crop = img[y0:y0+new_h, x0:x0+new_w]
    return crop

def align_rgb_to_thermal_grid(rgb_raw: np.ndarray, thermal_hw: tuple[int,int], zoom: float = 2.2) -> np.ndarray:
    """
    Align RGB to thermal domain by center-crop + zoom then resize to thermal size.
    thermal_hw: (H_th, W_th)
    """
    H_th, W_th = thermal_hw
    crop = center_crop_zoom(rgb_raw, zoom=zoom)
    aligned = cv2.resize(crop, (W_th, H_th), interpolation=cv2.INTER_AREA)
    return aligned

def build_patch_grid(H: int, W: int, patch: int, stride: int) -> pd.DataFrame:
    """Return dataframe grid with row,col,x0,y0,x1,y1. Based on thermal domain."""
    rows = ((H - patch) // stride) + 1
    cols = ((W - patch) // stride) + 1
    recs = []
    for r in range(rows):
        for c in range(cols):
            y0 = r * stride
            x0 = c * stride
            recs.append({
                "r": r, "c": c,
                "x0": x0, "y0": y0,
                "x1": x0 + patch,
                "y1": y0 + patch
            })
    return pd.DataFrame(recs), rows, cols

def extract_patch(img: np.ndarray, x0: int, y0: int, patch: int) -> np.ndarray:
    """Crop patch for either RGB (H,W,3) or thermal (H,W)."""
    if img.ndim == 3:
        return img[y0:y0+patch, x0:x0+patch, :]
    else:
        return img[y0:y0+patch, x0:x0+patch]

def safe_json_dump(obj, path: str | Path):
    with open(str(path), "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

print("✅ Helper functions loaded")


# Tahap 1 - Data Preprocessing & Preparation

## 1.1 Audit Pairing RGB–Thermal

In [ ]:
import os
from pathlib import Path
import pandas as pd

RAW_RGB_DIR = PROJECT_ROOT / "data" / "raw" / "rgb"
RAW_TH_DIR  = PROJECT_ROOT / "data" / "raw" / "thermal"

if not RAW_RGB_DIR.exists():
    raise FileNotFoundError(f"RAW_RGB_DIR tidak ditemukan: {RAW_RGB_DIR}")
if not RAW_TH_DIR.exists():
    raise FileNotFoundError(f"RAW_TH_DIR tidak ditemukan: {RAW_TH_DIR}")

rgb_exts = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")
rgb_paths = sorted([p for p in RAW_RGB_DIR.iterdir() if p.is_file() and p.name.endswith(rgb_exts)])
th_paths  = sorted([p for p in RAW_TH_DIR.iterdir() if p.is_file() and p.suffix.lower()==".npy"])

print("RAW_RGB_DIR:", RAW_RGB_DIR)
print("RAW_TH_DIR :", RAW_TH_DIR)
print("\n--- RAW COUNTS ---")
print("Total RGB files   :", len(rgb_paths))
print("Total Thermal npy :", len(th_paths))

def parse_frame_id_from_rgb_name(rgb_filename: str) -> str:
    """
    Konsisten dengan kode lama kamu:
    - buang ekstensi
    - kalau ada suffix '_V' di akhir stem -> buang
    Contoh:
    'DJI_..._0009_V.JPG' -> 'DJI_..._0009'
    """
    base = rgb_filename
    for ext in [".JPG", ".JPEG", ".PNG", ".jpg", ".jpeg", ".png"]:
        if base.endswith(ext):
            base = base[:-len(ext)]
            break
    if base.endswith("_V"):
        return base[:-2]
    return base

def thermal_name_from_frame_id(frame_id: str) -> str:
    return f"{frame_id}_T.npy"

rows = []
skipped_no_thermal = 0

for rgb_p in rgb_paths:
    frame_id = parse_frame_id_from_rgb_name(rgb_p.name)
    th_name  = thermal_name_from_frame_id(frame_id)
    th_p     = RAW_TH_DIR / th_name

    status = "OK_PAIR" if th_p.exists() else "NO_THERMAL"
    if status != "OK_PAIR":
        skipped_no_thermal += 1

    rows.append({
        "frame_id": frame_id,
        "rgb_path_raw": str(rgb_p),
        "thermal_path_raw": str(th_p) if th_p.exists() else "",
        "status": status
    })

audit_df = pd.DataFrame(rows)
n_ok = int((audit_df["status"]=="OK_PAIR").sum())
n_skip = int((audit_df["status"]!="OK_PAIR").sum())

print("\n--- PAIRING AUDIT SUMMARY ---")
print("RGB scanned        :", len(audit_df))
print("Valid pairs        :", n_ok)
print("Skipped no thermal :", n_skip)

display(audit_df.head(5))
display(audit_df[audit_df["status"]!="OK_PAIR"].head(5))


## 1.2 Alignment & Save FIX Frames

In [ ]:
import cv2
import numpy as np
import time

ensure_dir(FRAMES_DIR)
ensure_dir(RGB_ALIGNED_DIR)
ensure_dir(THERMAL_DIR)

pairs_df = audit_df[audit_df["status"]=="OK_PAIR"].copy().reset_index(drop=True)
if len(pairs_df) == 0:
    raise RuntimeError("Tidak ada pasangan valid untuk diproses. Cek naming _V dan _T.npy.")

ZOOM_USED = float(ZOOM_RGB_TO_THERMAL)

# sanity overlay output (opsional)
SANITY_DIR = OUT_DIR / "sanity_alignment"
ensure_dir(SANITY_DIR)
SANITY_N = 6
sanity_saved = 0

ok = 0
skipped_bad_read = 0
fail_log = []

t0 = time.time()

for i, row in pairs_df.iterrows():
    frame_id = row["frame_id"]
    rgb_raw_path = Path(row["rgb_path_raw"])
    th_raw_path  = Path(row["thermal_path_raw"])

    try:
        # load
        rgb_raw = read_rgb(rgb_raw_path)            # RGB uint8
        th      = read_thermal_npy(th_raw_path)     # float32 (H,W)
        H_th, W_th = th.shape[:2]

        # align
        rgb_aligned = align_rgb_to_thermal_grid(rgb_raw, (H_th, W_th), zoom=ZOOM_USED)

        # save (nama file DISAMAKAN dgn frame_id -> penting untuk tahap2 dst)
        out_rgb = RGB_ALIGNED_DIR / f"{frame_id}.png"
        out_th  = THERMAL_DIR / f"{frame_id}.npy"

        cv2.imwrite(str(out_rgb), cv2.cvtColor(rgb_aligned, cv2.COLOR_RGB2BGR))
        np.save(str(out_th), th.astype(np.float32))  # copy thermal, tetap grid thermal

        # update df
        pairs_df.loc[i, "rgb_path_aligned"] = str(out_rgb)
        pairs_df.loc[i, "thermal_path_fix"] = str(out_th)
        pairs_df.loc[i, "zoom_used"] = ZOOM_USED
        pairs_df.loc[i, "thermal_H"] = int(H_th)
        pairs_df.loc[i, "thermal_W"] = int(W_th)

        ok += 1

        # sanity save beberapa sample
        if sanity_saved < SANITY_N:
            th_vis = th.copy()
            lo, hi = np.nanpercentile(th_vis, 2), np.nanpercentile(th_vis, 98)
            th_vis = np.clip((th_vis - lo) / (hi - lo + 1e-6), 0, 1)
            th_u8  = (th_vis * 255).astype(np.uint8)
            th_cm  = cv2.applyColorMap(th_u8, cv2.COLORMAP_JET)
            th_cm  = cv2.cvtColor(th_cm, cv2.COLOR_BGR2RGB)

            side = np.concatenate([rgb_aligned, th_cm], axis=1)
            cv2.imwrite(str(SANITY_DIR / f"{frame_id}_rgb_vs_th.png"),
                        cv2.cvtColor(side, cv2.COLOR_RGB2BGR))
            sanity_saved += 1

    except Exception as e:
        skipped_bad_read += 1
        fail_log.append((frame_id, str(e)))
        continue

elapsed = time.time() - t0

pairs_df["status_fix"] = np.where(pairs_df["rgb_path_aligned"].astype(str)!="", "OK_FIX", "FAIL_FIX")

print("\n--- ALIGNMENT SUMMARY ---")
print("Zoom used            :", ZOOM_USED)
print("Pairs processed      :", len(pairs_df))
print("OK saved             :", ok)
print("Skipped bad read/load:", skipped_bad_read)
print("Sanity saved         :", sanity_saved, "->", SANITY_DIR)
print(f"Elapsed              : {elapsed:.1f}s")

if fail_log:
    print("\nFirst 5 fails:")
    for fr, err in fail_log[:5]:
        print("-", fr, "=>", err)


## 1.3 Save pairs_index.csv + Summary Statistik

In [ ]:
import json
import numpy as np
from pathlib import Path

def _exists(p):
    try:
        return Path(p).exists()
    except Exception:
        return False

pairs_df["rgb_exists"] = pairs_df["rgb_path_aligned"].apply(_exists)
pairs_df["th_exists"]  = pairs_df["thermal_path_fix"].apply(_exists)
pairs_df["status_fix"] = np.where(pairs_df["rgb_exists"] & pairs_df["th_exists"], "OK_FIX", "FAIL_FIX")

pairs_ok = pairs_df[pairs_df["status_fix"]=="OK_FIX"].copy().reset_index(drop=True)
pairs_fail = pairs_df[pairs_df["status_fix"]!="OK_FIX"].copy().reset_index(drop=True)

# simpan index (kolom ringkas tapi cukup untuk tahap 2 dst)
pairs_index = pairs_ok[[
    "frame_id",
    "rgb_path_raw", "thermal_path_raw",
    "rgb_path_aligned", "thermal_path_fix",
    "zoom_used", "thermal_H", "thermal_W"
]].copy()

pairs_index.to_csv(PAIRS_CSV, index=False)

# summary json
PREPROCESS_SUMMARY_JSON = FRAMES_DIR / "preprocess_summary.json"
summary = {
    "project_root": str(PROJECT_ROOT),
    "fix_root": str(FIX_ROOT),
    "raw_rgb_dir": str(RAW_RGB_DIR),
    "raw_thermal_dir": str(RAW_TH_DIR),
    "out_frames_dir": str(FRAMES_DIR),
    "rgb_aligned_dir": str(RGB_ALIGNED_DIR),
    "thermal_fix_dir": str(THERMAL_DIR),
    "pairs_index_csv": str(PAIRS_CSV),
    "counts": {
        "rgb_raw_total": int(len(audit_df)),
        "thermal_raw_total": int(len(list(RAW_TH_DIR.glob('*.npy')))),
        "pairs_valid_before_fix": int((audit_df["status"]=="OK_PAIR").sum()),
        "pairs_skipped_no_thermal": int((audit_df["status"]!="OK_PAIR").sum()),
        "pairs_fix_ok": int(len(pairs_ok)),
        "pairs_fix_fail": int(len(pairs_fail)),
    },
    "alignment": {
        "zoom_used": float(ZOOM_USED),
        "thermal_hw_example": [int(pairs_ok["thermal_H"].iloc[0]) if len(pairs_ok) else None,
                              int(pairs_ok["thermal_W"].iloc[0]) if len(pairs_ok) else None]
    }
}
safe_json_dump(summary, PREPROCESS_SUMMARY_JSON)

print("✅ Saved pairs_index.csv :", PAIRS_CSV)
print("✅ Saved summary.json    :", PREPROCESS_SUMMARY_JSON)

print("\n--- FINAL FIX STATS ---")
print("Valid FIX pairs :", len(pairs_ok))
print("Fail  FIX pairs :", len(pairs_fail))
print("RGB aligned files:", len(list(RGB_ALIGNED_DIR.glob("*.png"))))
print("Thermal npy files:", len(list(THERMAL_DIR.glob("*.npy"))))

display(pairs_index.head(5))


In [ ]:
print(rgb.shape, th.shape)


## 1.4 Qality Check

In [ ]:
# =========================================================
# FULL PIPELINE: load -> align (crop+zoom) -> overlay -> patches
# =========================================================

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 1) Utils
# -----------------------------
def normalize_to_u8(x, p1=1, p99=99):
    """Normalize 2D float array to uint8 for visualization."""
    x = x.astype(np.float32)
    lo, hi = np.nanpercentile(x, p1), np.nanpercentile(x, p99)
    x = np.clip((x - lo) / (hi - lo + 1e-6), 0, 1)
    return (x * 255).astype(np.uint8)

def center_crop(img, crop_h, crop_w):
    H, W = img.shape[:2]
    y0 = (H - crop_h) // 2
    x0 = (W - crop_w) // 2
    return img[y0:y0+crop_h, x0:x0+crop_w], (x0, y0, crop_w, crop_h)

def rgb_to_thermal_grid(rgb_bgr, th_shape_hw, zoom=2.2):
    """
    RGB wide -> crop center (zoom-in) -> resize to thermal grid.
    zoom bigger => crop smaller => RGB looks more zoomed-in.
    """
    Hth, Wth = th_shape_hw
    Hr, Wr = rgb_bgr.shape[:2]
    crop_h = int(Hr / zoom)
    crop_w = int(Wr / zoom)

    rgb_crop, roi = center_crop(rgb_bgr, crop_h, crop_w)
    rgb_aligned = cv2.resize(rgb_crop, (Wth, Hth), interpolation=cv2.INTER_LINEAR)
    return rgb_aligned, roi

def show_alignment(rgb_aligned_bgr, th_2d, title_suffix=""):
    """3-panel plot: RGB aligned, Thermal, Overlay."""
    th_u8 = normalize_to_u8(th_2d)
    rgb_show = cv2.cvtColor(rgb_aligned_bgr, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 4))
    plt.subplot(1,3,1); plt.imshow(rgb_show); plt.title(f"RGB aligned{title_suffix}"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(th_u8, cmap="inferno"); plt.title(f"Thermal{title_suffix}"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(rgb_show); plt.imshow(th_u8, cmap="inferno", alpha=0.35); plt.title(f"Overlay{title_suffix}"); plt.axis("off")
    plt.tight_layout()
    plt.show()

def extract_paired_patches(rgb_aligned_bgr, th_2d, patch=128, stride=128):
    """
    Extract paired patches (RGB + Thermal) from same grid.
    Return:
      rgb_patches: (N, patch, patch, 3) uint8
      th_patches : (N, patch, patch) float32
      meta       : list dict per patch
    """
    H, W = th_2d.shape[:2]
    rgb_patches, th_patches, meta = [], [], []

    for y in range(0, H - patch + 1, stride):
        for x in range(0, W - patch + 1, stride):
            rgb_p = rgb_aligned_bgr[y:y+patch, x:x+patch]
            th_p  = th_2d[y:y+patch, x:x+patch]

            rgb_patches.append(rgb_p)
            th_patches.append(th_p.astype(np.float32))

            meta.append({
                "x": x, "y": y,
                "patch": patch,
                "row": y // stride,
                "col": x // stride
            })

    return np.stack(rgb_patches, axis=0), np.stack(th_patches, axis=0), meta


# -----------------------------
# 2) Paths (EDIT THIS)
# -----------------------------
rgb_path = "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/data/raw/rgb/DJI_20251223095741_0048_V.JPG"
th_path  = "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/data/raw/thermal/DJI_20251223095741_0048_T.npy"

zoom = 2.2       # <-- ganti kalau perlu (2.0 / 2.2 / 2.4 / 2.6)
patch = 128
stride = 128


# -----------------------------
# 3) Load data
# -----------------------------
rgb = cv2.imread(rgb_path)   # BGR
if rgb is None:
    raise ValueError(f"RGB gagal dibaca. Cek path: {rgb_path}")

th = np.load(th_path)
if th.ndim != 2:
    raise ValueError(f"Thermal npy harus 2D. Dapat shape={th.shape}")

print("RGB shape:", rgb.shape, rgb.dtype)
print("TH  shape:", th.shape, th.dtype)


# -----------------------------
# 4) Align RGB -> thermal grid
# -----------------------------
rgb_aligned, roi = rgb_to_thermal_grid(rgb, th.shape[:2], zoom=zoom)
print("ALIGN PARAMS:", {"zoom": zoom, "roi": roi})
print("RGB aligned shape:", rgb_aligned.shape)


# -----------------------------
# 5) Sanity check overlay
# -----------------------------
show_alignment(rgb_aligned, th, title_suffix=f" (zoom={zoom})")


# -----------------------------
# 6) Extract paired patches
# -----------------------------
rgb_patches, th_patches, meta = extract_paired_patches(rgb_aligned, th, patch=patch, stride=stride)
print("RGB patches:", rgb_patches.shape, rgb_patches.dtype)
print("TH  patches:", th_patches.shape, th_patches.dtype)
print("Example meta[0]:", meta[0])


# -----------------------------
# 7) (Optional) Visualize a few random paired patches
# -----------------------------
idxs = [0, len(meta)//2, len(meta)-1] if len(meta) >= 3 else list(range(len(meta)))

plt.figure(figsize=(12, 6))
for i, idx in enumerate(idxs, 1):
    rgb_p = cv2.cvtColor(rgb_patches[idx], cv2.COLOR_BGR2RGB)
    th_u8 = normalize_to_u8(th_patches[idx])

    ax = plt.subplot(2, len(idxs), i)
    ax.imshow(rgb_p); ax.set_title(f"RGB patch #{idx}"); ax.axis("off")

    ax = plt.subplot(2, len(idxs), i + len(idxs))
    ax.imshow(th_u8, cmap="inferno"); ax.set_title(f"TH patch #{idx}"); ax.axis("off")

plt.tight_layout()
plt.show()


# TAHAP 2 — Ekstraksi Patch & Konstruksi Dataset


## 2.1 Definisi Patch Grid

In [ ]:
import pandas as pd
import numpy as np

if not PAIRS_CSV.exists():
    raise FileNotFoundError(f"pairs_index.csv tidak ditemukan: {PAIRS_CSV}. Jalankan TAHAP 1 dulu.")

pairs_index = pd.read_csv(PAIRS_CSV)
print("✅ Loaded pairs_index.csv :", PAIRS_CSV)
print("Total FIX pairs:", len(pairs_index))

# Ambil thermal size (asumsi konsisten)
H_th = int(pairs_index["thermal_H"].iloc[0]) if "thermal_H" in pairs_index.columns else None
W_th = int(pairs_index["thermal_W"].iloc[0]) if "thermal_W" in pairs_index.columns else None

# Jika kolom thermal_H/W tidak ada (index lama), infer dari 1 file thermal
if H_th is None or W_th is None:
    sample_th = read_thermal_npy(pairs_index["thermal_path_fix"].iloc[0])
    H_th, W_th = sample_th.shape[:2]

print("Thermal reference HW:", (H_th, W_th))

patch = int(PATCH_SIZE)
stride = int(STRIDE)

patch_grid_df, n_rows, n_cols = build_patch_grid(H_th, W_th, patch=patch, stride=stride)

print("\n--- PATCH GRID SUMMARY ---")
print("PATCH_SIZE:", patch, "| STRIDE:", stride)
print("Rows:", n_rows, "| Cols:", n_cols, "| Patches per frame:", len(patch_grid_df))

# save grid json (untuk deploy juga)
ensure_dir(PATCH_META_DIR)
grid_info = {
    "thermal_hw": [H_th, W_th],
    "patch": patch,
    "stride": stride,
    "rows": n_rows,
    "cols": n_cols,
    "patches_per_frame": int(len(patch_grid_df))
}
safe_json_dump(grid_info, PATCH_GRID_JSON)

print("✅ Saved patch_grid.json:", PATCH_GRID_JSON)
display(patch_grid_df.head(10))


## 2.2 Extract Paired Patches

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import time

# --- toggle ---
SAVE_PATCHES = True  # ✅ untuk kamu ini masuk akal (total patch kecil); ganti False kalau mau hemat I/O

ensure_dir(PATCH_ROOT)
ensure_dir(PATCH_META_DIR)
if SAVE_PATCHES:
    ensure_dir(PATCH_RGB_DIR)
    ensure_dir(PATCH_TH_DIR)

pairs_index = pd.read_csv(PAIRS_CSV)
needed_cols = ["frame_id", "rgb_path_aligned", "thermal_path_fix"]
for c in needed_cols:
    if c not in pairs_index.columns:
        raise ValueError(f"Kolom '{c}' tidak ada di pairs_index.csv. Kolom tersedia: {list(pairs_index.columns)}")

t0 = time.time()

rows = []
fail = 0
ok_frames = 0
total_patches = 0

for _, r in pairs_index.iterrows():
    frame_id = str(r["frame_id"])
    rgb_path = Path(r["rgb_path_aligned"])
    th_path  = Path(r["thermal_path_fix"])

    if (not rgb_path.exists()) or (not th_path.exists()):
        fail += 1
        continue

    try:
        rgb = read_rgb(rgb_path)          # (H,W,3) RGB aligned
        th  = read_thermal_npy(th_path)   # (H,W) float32

        # sanity: thermal domain should match rgb aligned size
        if rgb.shape[0] != th.shape[0] or rgb.shape[1] != th.shape[1]:
            raise ValueError(f"Size mismatch rgb {rgb.shape[:2]} vs th {th.shape[:2]} for {frame_id}")

        H, W = th.shape[:2]

        # build grid once per frame (but using same params)
        grid_df, rr, cc = build_patch_grid(H, W, patch=PATCH_SIZE, stride=STRIDE)

        for j, g in grid_df.iterrows():
            r0, c0 = int(g["r"]), int(g["c"])
            x0, y0 = int(g["x0"]), int(g["y0"])

            patch_id = f"{frame_id}_r{r0:03d}_c{c0:03d}"

            # extract
            rgb_patch = extract_patch(rgb, x0, y0, PATCH_SIZE)
            th_patch  = extract_patch(th,  x0, y0, PATCH_SIZE)

            # optional save patch to disk
            rgb_patch_path = ""
            th_patch_path  = ""
            if SAVE_PATCHES:
                rgb_patch_path = str(PATCH_RGB_DIR / f"{patch_id}.png")
                th_patch_path  = str(PATCH_TH_DIR  / f"{patch_id}.npy")

                cv2.imwrite(rgb_patch_path, cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2BGR))
                np.save(th_patch_path, th_patch.astype(np.float32))

            rows.append({
                "patch_id": patch_id,
                "frame_id": frame_id,
                "r": r0,
                "c": c0,
                "x0": x0,
                "y0": y0,
                "patch": int(PATCH_SIZE),
                "stride": int(STRIDE),

                # source frame paths (selalu disimpan)
                "rgb_frame_path": str(rgb_path),
                "thermal_frame_path": str(th_path),

                # optional patch paths
                "rgb_patch_path": rgb_patch_path,
                "thermal_patch_path": th_patch_path,
            })

            total_patches += 1

        ok_frames += 1

    except Exception as e:
        fail += 1
        continue

patches_all = pd.DataFrame(rows)
elapsed = time.time() - t0

print("\n--- PATCH EXTRACTION SUMMARY ---")
print("SAVE_PATCHES:", SAVE_PATCHES)
print("Frames OK    :", ok_frames)
print("Frames FAIL  :", fail)
print("Total patches:", len(patches_all))
print(f"Elapsed      : {elapsed:.1f}s")

display(patches_all.head(5))


## 2.3 Save patches_all.csv + Summary

In [ ]:
import numpy as np
from pathlib import Path

ensure_dir(PATCH_META_DIR)

# basic checks
if len(patches_all) == 0:
    raise RuntimeError("patches_all kosong. Cek pairs_index.csv dan folder frames/ output Tahap 1.")

# Save CSV
patches_all.to_csv(PATCHES_ALL_CSV, index=False)

# summary
patch_extract_summary = {
    "pairs_index_csv": str(PAIRS_CSV),
    "patches_all_csv": str(PATCHES_ALL_CSV),
    "save_patches": bool(SAVE_PATCHES),
    "patch": int(PATCH_SIZE),
    "stride": int(STRIDE),
    "counts": {
        "frames_in_index": int(len(pd.read_csv(PAIRS_CSV))),
        "patch_rows": int(len(patches_all)),
        "unique_frames": int(patches_all["frame_id"].nunique()),
        "patches_per_frame_mode": int(patches_all.groupby("frame_id").size().mode().iloc[0]),
    },
    "paths": {
        "patch_root": str(PATCH_ROOT),
        "patch_rgb_dir": str(PATCH_RGB_DIR) if SAVE_PATCHES else "",
        "patch_th_dir": str(PATCH_TH_DIR) if SAVE_PATCHES else "",
        "patch_meta_dir": str(PATCH_META_DIR),
        "patch_grid_json": str(PATCH_GRID_JSON)
    }
}

PATCH_SUMMARY_JSON = PATCH_META_DIR / "patch_extract_summary.json"
safe_json_dump(patch_extract_summary, PATCH_SUMMARY_JSON)

print("✅ Saved patches_all.csv         :", PATCHES_ALL_CSV)
print("✅ Saved patch_extract_summary.json:", PATCH_SUMMARY_JSON)

print("\n--- QUICK STATS ---")
print("Unique frames :", patches_all['frame_id'].nunique())
print("Total patches :", len(patches_all))
print("Label columns present? (belum) ->", any(c.startswith("label") or c=="y" for c in patches_all.columns))

# Optional: show patch path completeness
if SAVE_PATCHES:
    n_rgb_paths = int((patches_all["rgb_patch_path"].astype(str) != "").sum())
    n_th_paths  = int((patches_all["thermal_patch_path"].astype(str) != "").sum())
    print("Saved RGB patch files rows:", n_rgb_paths)
    print("Saved TH  patch files rows:", n_th_paths)

display(patches_all.sample(5, random_state=SEED))


In [ ]:
patches_all.head()


# TAHAP 3 — Feature Extraction & Labeling

## 3.1 Template Pool Labeling Manual

In [ ]:
import pandas as pd
import numpy as np

if not PATCHES_ALL_CSV.exists():
    raise FileNotFoundError(f"patches_all.csv tidak ditemukan: {PATCHES_ALL_CSV}. Jalankan TAHAP 2 dulu.")

ensure_dir(LABEL_GOLD_DIR)

patches_all = pd.read_csv(PATCHES_ALL_CSV)
print("✅ Loaded patches_all.csv:", PATCHES_ALL_CSV)
print("Rows:", len(patches_all), "| Unique frames:", patches_all["frame_id"].nunique())

# --- konfigurasi pool manual ---
POOL_SIZE = 2000  # target pool manual (ubah sesuai kebutuhan)
PER_FRAME_CAP = 20  # batasi max patch per frame agar tidak bias 1 frame

# pastikan ada path rgb patch untuk memudahkan anotasi
# kalau SAVE_PATCHES=False di Tahap 2, kita buat "virtual view path" dari rgb_frame_path + coords
has_rgb_patch_files = "rgb_patch_path" in patches_all.columns and (patches_all["rgb_patch_path"].astype(str) != "").any()

df = patches_all.copy()

# sampling: balanced coverage by frame
pool_rows = []
rng = np.random.default_rng(SEED)

for frame_id, g in df.groupby("frame_id"):
    g = g.sample(min(len(g), PER_FRAME_CAP), random_state=SEED)
    pool_rows.append(g)

pool_df = pd.concat(pool_rows, ignore_index=True)

# kalau masih lebih besar dari POOL_SIZE -> sample lagi
if len(pool_df) > POOL_SIZE:
    pool_df = pool_df.sample(POOL_SIZE, random_state=SEED).reset_index(drop=True)
else:
    pool_df = pool_df.reset_index(drop=True)

# template columns (manual annotation)
template = pd.DataFrame({
    "patch_id": pool_df["patch_id"].astype(str),
    "frame_id": pool_df["frame_id"].astype(str),
    "r": pool_df["r"].astype(int),
    "c": pool_df["c"].astype(int),
    "x0": pool_df["x0"].astype(int),
    "y0": pool_df["y0"].astype(int),
    # untuk anotasi cepat: pakai patch file kalau ada, kalau tidak pakai frame path (nanti dipotong on-the-fly)
    "rgb_view_path": pool_df["rgb_patch_path"].astype(str) if has_rgb_patch_files else pool_df["rgb_frame_path"].astype(str),
    "label_manual": "",          # isi: 0 atau 1
    "manual_conf": "",           # opsional: low/med/high atau 0-1
    "annotator": "",             # nama/initial
    "notes": ""                  # catatan
})

LABEL_GOLD_TEMPLATE = LABEL_GOLD_DIR / "labels_gold_template.csv"
template.to_csv(LABEL_GOLD_TEMPLATE, index=False)

print("\n--- GOLD POOL SUMMARY ---")
print("Template saved:", LABEL_GOLD_TEMPLATE)
print("Pool size:", len(template))
print("Has rgb patch files:", has_rgb_patch_files)

display(template.head(10))


## 3.2 — UI Manual Labeling (RGB-only)

In [ ]:
# ============================================
# TAHAP 3.2 — UI Manual Labeling (GOLD) [autosave + resume]
#   Input : labels_gold_template.csv
#   Output: labels_gold.csv
#   Buttons: Back | Save & Next | Skip | Cancel
#   Columns output (minimal): patch_id, label_manual, notes
# ============================================

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

LABEL_TEMPLATE = LABEL_GOLD_DIR / "labels_gold_template.csv"
LABEL_GOLD_OUT = LABEL_GOLD_DIR / "labels_gold.csv"

if not LABEL_TEMPLATE.exists():
    raise FileNotFoundError(f"Template tidak ditemukan: {LABEL_TEMPLATE}. Jalankan TAHAP 3.1 dulu.")

pool = pd.read_csv(LABEL_TEMPLATE)
pool["patch_id"] = pool["patch_id"].astype(str)

# Output minimal sesuai preferensi kamu
OUT_COLS = ["patch_id", "label_manual", "notes"]

if not LABEL_GOLD_OUT.exists():
    pd.DataFrame(columns=OUT_COLS).to_csv(LABEL_GOLD_OUT, index=False)

done = pd.read_csv(LABEL_GOLD_OUT)
done_ids = set(done["patch_id"].astype(str).tolist()) if len(done) else set()

before = len(pool)
pool = pool[~pool["patch_id"].isin(done_ids)].reset_index(drop=True)

print("✅ Template:", LABEL_TEMPLATE, "| rows:", before)
print("✅ Output  :", LABEL_GOLD_OUT)
print("Already labeled:", len(done_ids))
print("Remaining:", len(pool))

idx = 0
total = len(pool)
is_cancelled = False

title = widgets.HTML()
img_out = widgets.Output()

# Rubric (opsional)
c1 = widgets.Checkbox(value=False, description="K1: warna stres (pucat/kekuningan/tidak merata)")
c2 = widgets.Checkbox(value=False, description="K2: kanopi tidak seragam (bercak/patchy)")
c3 = widgets.Checkbox(value=False, description="K3: kanopi jarang (celah banyak)")
c4 = widgets.Checkbox(value=False, description="K4: layu/kerusakan (daun abnormal/terkulai)")

score_lbl = widgets.HTML()
auto_lbl  = widgets.HTML()

radio = widgets.ToggleButtons(
    options=[("Normal (0)", 0), ("Stress (1)", 1)],
    value=0,
    description="Label:"
)

note  = widgets.Text(value="", placeholder="catatan opsional (blur, non-padi, ragu)", description="Note:")

# Buttons sesuai request
btn_back  = widgets.Button(description="Back")
btn_save  = widgets.Button(description="Save & Next", button_style="success")
btn_skip  = widgets.Button(description="Skip", button_style="warning")
btn_cancel= widgets.Button(description="Cancel", button_style="danger")

msg = widgets.HTML()

def compute_score():
    score = int(c1.value) + int(c2.value) + int(c3.value) + int(c4.value)
    auto  = 1 if score >= 2 else 0
    return score, auto

def reset_inputs():
    c1.value=False; c2.value=False; c3.value=False; c4.value=False
    note.value=""
    radio.value=0
    msg.value=""

def append_and_autosave(rec):
    df_old = pd.read_csv(LABEL_GOLD_OUT)
    df_new = pd.concat([df_old, pd.DataFrame([rec])], ignore_index=True)
    # jaga kolom
    for c in OUT_COLS:
        if c not in df_new.columns:
            df_new[c] = ""
    df_new = df_new[OUT_COLS]
    df_new.to_csv(LABEL_GOLD_OUT, index=False)

def render():
    global idx, total, is_cancelled
    clear_output(wait=True)

    if is_cancelled:
        display(widgets.HTML(f"<b>Dibatalkan.</b> Data aman tersimpan di: <code>{LABEL_GOLD_OUT}</code>"))
        return

    if total == 0:
        display(widgets.HTML("<b>Selesai.</b> Tidak ada patch tersisa (atau semuanya sudah dilabel)."))
        display(widgets.HTML(f"Output: <code>{LABEL_GOLD_OUT}</code>"))
        return

    idx = max(0, min(idx, total-1))
    row = pool.iloc[idx]

    patch_id = str(row["patch_id"])
    frame_id = str(row.get("frame_id",""))
    r = row.get("r",""); c = row.get("c","")
    p = str(row["rgb_view_path"])

    score, auto = compute_score()
    title.value = f"""
    <b>GOLD Labeling {idx+1}/{total}</b><br>
    patch_id: <code>{patch_id}</code><br>
    frame_id: <code>{frame_id}</code> | r,c=({r},{c})
    """

    score_lbl.value = f"<b>Rubric score:</b> {score} (auto>=2 → Stress={auto})"
    auto_lbl.value  = f"<b>Rubric suggestion:</b> <span style='font-size:16px'>{'Stress' if auto==1 else 'Normal'}</span>"

    img_out.clear_output()
    with img_out:
        try:
            im = Image.open(p).convert("RGB")
            display(im)
        except Exception as e:
            display(widgets.HTML(f"<b style='color:red'>Gagal load image:</b><br><code>{p}</code><br>{e}"))

    display(title)
    display(img_out)
    display(widgets.VBox([c1,c2,c3,c4,score_lbl,auto_lbl]))
    display(widgets.VBox([radio, note]))
    display(widgets.HBox([btn_back, btn_save, btn_skip, btn_cancel]))
    display(msg)
    display(widgets.HTML(f"<small>Autosave ke Drive: <code>{LABEL_GOLD_OUT}</code></small>"))

def on_save(_):
    global idx
    if total == 0:
        return

    row = pool.iloc[idx]
    patch_id = str(row["patch_id"])

    rec = {
        "patch_id": patch_id,
        "label_manual": int(radio.value),
        "notes": note.value.strip()
    }
    append_and_autosave(rec)
    msg.value = "<span style='color:green'><b>Saved.</b> (autosaved)</span>"
    idx += 1
    reset_inputs()
    render()

def on_skip(_):
    global idx
    if total == 0:
        return

    row = pool.iloc[idx]
    patch_id = str(row["patch_id"])
    n = note.value.strip()

    rec = {
        "patch_id": patch_id,
        "label_manual": "",  # kosong => dianggap tidak dilabel
        "notes": ("[SKIP] " + n).strip() if n else "[SKIP]"
    }
    append_and_autosave(rec)
    msg.value = "<span style='color:orange'><b>Skipped.</b> (autosaved)</span>"
    idx += 1
    reset_inputs()
    render()

def on_back(_):
    global idx
    idx -= 1
    reset_inputs()
    render()

def on_cancel(_):
    global is_cancelled
    is_cancelled = True
    render()

btn_save.on_click(on_save)
btn_skip.on_click(on_skip)
btn_back.on_click(on_back)
btn_cancel.on_click(on_cancel)

render()


In [ ]:
import shutil, time
from pathlib import Path

LABEL_GOLD_OUT = FIX_ROOT / "labels" / "gold" / "labels_gold.csv"
bak = LABEL_GOLD_OUT.parent / f"labels_gold_BACKUP_{time.strftime('%Y%m%d_%H%M%S')}.csv"
shutil.copy2(LABEL_GOLD_OUT, bak)
print("✅ Backup saved:", bak)


In [ ]:
import pandas as pd

LABEL_TEMPLATE = FIX_ROOT / "labels" / "gold" / "labels_gold_template.csv"
LABEL_GOLD_OUT = FIX_ROOT / "labels" / "gold" / "labels_gold.csv"

N_RESET = 700

tpl = pd.read_csv(LABEL_TEMPLATE)
gold = pd.read_csv(LABEL_GOLD_OUT)

# patch_id yg mau diulang = 1..700 sesuai urutan template
reset_ids = set(tpl["patch_id"].astype(str).iloc[:N_RESET].tolist())

before = len(gold)
gold["patch_id"] = gold["patch_id"].astype(str)

# hapus label untuk patch_id tsb
gold2 = gold[~gold["patch_id"].isin(reset_ids)].copy()

after = len(gold2)
gold2.to_csv(LABEL_GOLD_OUT, index=False)

print("✅ RESET DONE")
print("Template:", LABEL_TEMPLATE)
print("Labels  :", LABEL_GOLD_OUT)
print("Reset N patch:", len(reset_ids))
print("Rows before:", before, "| after:", after, "| removed:", before-after)


In [ ]:
# ============================================
# TAHAP 3.2R — UI Manual Labeling (RE-LABEL 1–700)
#   Purpose : ulang labeling untuk template index 1..700 (urut template)
#   Input   : labels_gold_template.csv  (sumber urutan)
#   Output  : labels_gold_relabel_1_700.csv  (BARU, urut 1..700)
#   Notes   : autosave + resume + tombol Back/Save/Skip/Cancel
# ============================================

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# ---- CONFIG ----
LABEL_TEMPLATE = LABEL_GOLD_DIR / "labels_gold_template.csv"
LABEL_BASE_GOLD = LABEL_GOLD_DIR / "labels_gold.csv"  # hanya untuk info/cek (optional)
LABEL_OUT = LABEL_GOLD_DIR / "labels_gold_relabel_1_700.csv"

N_RELAB = 700  # relabel template #1..#700 (1-based)

if not LABEL_TEMPLATE.exists():
    raise FileNotFoundError(f"Template tidak ditemukan: {LABEL_TEMPLATE}. Jalankan TAHAP 3.1 dulu.")

tpl = pd.read_csv(LABEL_TEMPLATE)
tpl["patch_id"] = tpl["patch_id"].astype(str)

# Ambil hanya 1..700 sesuai urutan template
pool = tpl.iloc[:N_RELAB].copy().reset_index(drop=True)

# Output minimal sesuai preferensi kamu
OUT_COLS = ["patch_id", "label_manual", "notes"]

# ----- RESUME LOGIC -----
# Kalau file output belum ada, buat baru (kosong)
if not LABEL_OUT.exists():
    pd.DataFrame(columns=OUT_COLS).to_csv(LABEL_OUT, index=False)

done = pd.read_csv(LABEL_OUT)
done["patch_id"] = done["patch_id"].astype(str) if len(done) else done.get("patch_id", pd.Series([], dtype=str))
done_ids = set(done["patch_id"].tolist()) if len(done) else set()

# Filter patch yang sudah dilabel di sesi relabel ini
pool = pool[~pool["patch_id"].isin(done_ids)].reset_index(drop=True)

print("✅ TEMPLATE:", LABEL_TEMPLATE)
print("✅ OUTPUT  :", LABEL_OUT, "(file relabel khusus)")
print("Relabel target:", N_RELAB, "patch (template #1..#700)")
print("Already relabeled:", len(done_ids))
print("Remaining:", len(pool))
if len(pool) > 0:
    print("First remaining:", pool.iloc[0]["patch_id"])
    print("Last remaining :", pool.iloc[-1]["patch_id"])

# ---- UI STATE ----
idx = 0
total = len(pool)
is_cancelled = False

title = widgets.HTML()
img_out = widgets.Output()

# Rubric (opsional, bantu konsistensi)
c1 = widgets.Checkbox(value=False, description="K1: warna stres (pucat/kekuningan/tidak merata)")
c2 = widgets.Checkbox(value=False, description="K2: kanopi tidak seragam (bercak/patchy)")
c3 = widgets.Checkbox(value=False, description="K3: kanopi jarang (celah banyak)")
c4 = widgets.Checkbox(value=False, description="K4: layu/kerusakan (daun abnormal/terkulai)")

score_lbl = widgets.HTML()
auto_lbl  = widgets.HTML()

radio = widgets.ToggleButtons(
    options=[("Normal (0)", 0), ("Stress (1)", 1)],
    value=0,
    description="Label:"
)

note  = widgets.Text(value="", placeholder="catatan opsional (blur, non-padi, ragu)", description="Note:")

# Buttons
btn_back   = widgets.Button(description="Back")
btn_save   = widgets.Button(description="Save & Next", button_style="success")
btn_skip   = widgets.Button(description="Skip", button_style="warning")
btn_cancel = widgets.Button(description="Cancel", button_style="danger")

msg = widgets.HTML()

def compute_score():
    score = int(c1.value) + int(c2.value) + int(c3.value) + int(c4.value)
    auto  = 1 if score >= 2 else 0
    return score, auto

def reset_inputs():
    c1.value=False; c2.value=False; c3.value=False; c4.value=False
    note.value=""
    radio.value=0
    msg.value=""

def append_and_autosave(rec):
    df_old = pd.read_csv(LABEL_OUT)
    df_new = pd.concat([df_old, pd.DataFrame([rec])], ignore_index=True)

    # jaga kolom (minimal)
    for c in OUT_COLS:
        if c not in df_new.columns:
            df_new[c] = ""
    df_new = df_new[OUT_COLS]
    df_new.to_csv(LABEL_OUT, index=False)

def render():
    global idx, total, is_cancelled
    clear_output(wait=True)

    if is_cancelled:
        display(widgets.HTML(f"<b>Dibatalkan.</b> Data aman tersimpan di: <code>{LABEL_OUT}</code>"))
        return

    if total == 0:
        display(widgets.HTML("<b>Selesai.</b> Tidak ada patch tersisa untuk relabel 1–700 (semua sudah dilabel ulang)."))
        display(widgets.HTML(f"Output: <code>{LABEL_OUT}</code>"))
        # info tambahan (optional)
        if LABEL_BASE_GOLD.exists():
            display(widgets.HTML(f"<small>Catatan: file label utama tetap: <code>{LABEL_BASE_GOLD}</code></small>"))
        return

    idx = max(0, min(idx, total-1))
    row = pool.iloc[idx]

    patch_id = str(row["patch_id"])
    frame_id = str(row.get("frame_id",""))
    r = row.get("r",""); c = row.get("c","")
    p = str(row.get("rgb_view_path",""))

    score, auto = compute_score()
    title.value = f"""
    <b>RE-LABEL 1–700: {idx+1}/{total}</b><br>
    patch_id: <code>{patch_id}</code><br>
    frame_id: <code>{frame_id}</code> | r,c=({r},{c})
    """

    score_lbl.value = f"<b>Rubric score:</b> {score} (auto>=2 → Stress={auto})"
    auto_lbl.value  = f"<b>Rubric suggestion:</b> <span style='font-size:16px'>{'Stress' if auto==1 else 'Normal'}</span>"

    img_out.clear_output()
    with img_out:
        try:
            im = Image.open(p).convert("RGB")
            display(im)
        except Exception as e:
            display(widgets.HTML(f"<b style='color:red'>Gagal load image:</b><br><code>{p}</code><br>{e}"))

    display(title)
    display(img_out)
    display(widgets.VBox([c1,c2,c3,c4,score_lbl,auto_lbl]))
    display(widgets.VBox([radio, note]))
    display(widgets.HBox([btn_back, btn_save, btn_skip, btn_cancel]))
    display(msg)
    display(widgets.HTML(f"<small>Autosave ke Drive: <code>{LABEL_OUT}</code></small>"))

def on_save(_):
    global idx
    if total == 0:
        return

    row = pool.iloc[idx]
    patch_id = str(row["patch_id"])

    rec = {
        "patch_id": patch_id,
        "label_manual": int(radio.value),
        "notes": note.value.strip()
    }
    append_and_autosave(rec)
    msg.value = "<span style='color:green'><b>Saved.</b> (autosaved)</span>"
    idx += 1
    reset_inputs()
    render()

def on_skip(_):
    global idx
    if total == 0:
        return

    row = pool.iloc[idx]
    patch_id = str(row["patch_id"])
    n = note.value.strip()

    rec = {
        "patch_id": patch_id,
        "label_manual": "",  # kosong => dianggap tidak dilabel (skip)
        "notes": ("[SKIP] " + n).strip() if n else "[SKIP]"
    }
    append_and_autosave(rec)
    msg.value = "<span style='color:orange'><b>Skipped.</b> (autosaved)</span>"
    idx += 1
    reset_inputs()
    render()

def on_back(_):
    global idx
    idx -= 1
    reset_inputs()
    render()

def on_cancel(_):
    global is_cancelled
    is_cancelled = True
    render()

btn_save.on_click(on_save)
btn_skip.on_click(on_skip)
btn_back.on_click(on_back)
btn_cancel.on_click(on_cancel)

render()


In [ ]:
# ============================================
# MERGE RELABEL 1–700 -> labels_gold.csv
# + SORT hasil akhir mengikuti urutan labels_gold_template.csv
# ============================================

import pandas as pd
import shutil, time
from pathlib import Path

LABEL_TEMPLATE = LABEL_GOLD_DIR / "labels_gold_template.csv"
LABEL_MAIN     = LABEL_GOLD_DIR / "labels_gold.csv"
LABEL_RELAB    = LABEL_GOLD_DIR / "labels_gold_relabel_1_700.csv"

N_RELAB = 700  # harus sama seperti sesi relabel

# ---- cek file ----
for p in [LABEL_TEMPLATE, LABEL_MAIN, LABEL_RELAB]:
    if not p.exists():
        raise FileNotFoundError(f"File tidak ditemukan: {p}")

# ---- backup labels_gold.csv dulu ----
bak = LABEL_MAIN.parent / f"labels_gold_BACKUP_before_merge_{time.strftime('%Y%m%d_%H%M%S')}.csv"
shutil.copy2(LABEL_MAIN, bak)
print("✅ Backup saved:", bak)

# ---- load ----
tpl = pd.read_csv(LABEL_TEMPLATE)
tpl["patch_id"] = tpl["patch_id"].astype(str)

main = pd.read_csv(LABEL_MAIN)
main["patch_id"] = main["patch_id"].astype(str)

relab = pd.read_csv(LABEL_RELAB)
relab["patch_id"] = relab["patch_id"].astype(str)

# ---- patch_id target = template #1..#700 ----
target_ids = set(tpl["patch_id"].iloc[:N_RELAB].tolist())

# ---- validasi: relabel harus mencakup patch target ----
relab_ids = set(relab["patch_id"].tolist())
missing = sorted(list(target_ids - relab_ids))
if len(missing) > 0:
    print("⚠️ WARNING: Ada patch target yang belum ada di relabel file (belum kelabel / belum tersentuh):", len(missing))
    print("Contoh missing:", missing[:5])
    print("Tetap lanjut merge, tapi patch missing akan tetap memakai label lama (kalau ada) / kosong.")
else:
    print("✅ Relabel coverage OK: semua patch 1–700 ada di relabel file.")

# ---- samakan kolom: pakai kolom minimal yang kamu pakai (patch_id, label_manual, notes)
# Jika di main ada kolom lain, biarkan tetap ada (kita merge kolom minimal lalu pertahankan sisanya).
needed_cols = ["patch_id", "label_manual", "notes"]
for c in needed_cols:
    if c not in main.columns:
        main[c] = ""
    if c not in relab.columns:
        relab[c] = ""

# ---- 1) buang dulu record 1–700 dari main ----
before_main = len(main)
main_keep = main[~main["patch_id"].isin(target_ids)].copy()
removed = before_main - len(main_keep)

# ---- 2) gabungkan main_keep + relab (relab jadi sumber terbaru untuk 1–700) ----
merged = pd.concat([main_keep, relab[needed_cols]], ignore_index=True)

# ---- 3) jika ada duplikat patch_id (harusnya tidak), keep yang terakhir (relab) ----
merged = merged.drop_duplicates(subset=["patch_id"], keep="last").copy()

print("\n--- MERGE SUMMARY ---")
print("Main rows before :", before_main)
print("Removed 1–700     :", removed)
print("Relabel rows add  :", len(relab))
print("Merged rows now   :", len(merged))

# ---- 4) Sort sesuai urutan template ----
# Buat mapping patch_id -> urutan di template
tpl_order = {pid: i for i, pid in enumerate(tpl["patch_id"].astype(str).tolist())}

def order_key(pid: str):
    # patch yang ada di template akan diurutkan sesuai template
    # patch yang tidak ada di template taruh di bawah (akhir)
    return tpl_order.get(pid, 10**12)

merged["_order"] = merged["patch_id"].astype(str).map(order_key)

# sort by template order
merged_sorted = merged.sort_values(["_order", "patch_id"], ascending=[True, True]).drop(columns=["_order"])

# ---- 5) simpan balik ke labels_gold.csv ----
merged_sorted.to_csv(LABEL_MAIN, index=False)

print("\n✅ DONE: labels_gold.csv sudah digabung + diurutkan sesuai template")
print("Saved:", LABEL_MAIN)

# sanity check: print first 5, last 5 patch_id
print("\nFirst 5 patch_id in labels_gold.csv:")
print(merged_sorted["patch_id"].head(5).tolist())
print("\nLast 5 patch_id in labels_gold.csv:")
print(merged_sorted["patch_id"].tail(5).tolist())


## 3.3 — Canopy Masking & Thermal Feature Prep

In [ ]:
# ============================================
# TAHAP 3.3 — Canopy Masking & Thermal Feature Prep
#   SAFE VERSION: progress + autosave + resume
# ============================================

import numpy as np
import pandas as pd
import cv2
from pathlib import Path
from tqdm import tqdm

THERM_DIR = FIX_ROOT / "features" / "thermal"
ensure_dir(THERM_DIR)

THERM_FEAT_CSV = THERM_DIR / "thermal_features.csv"
PATCHES_ENRICHED_CSV = FIX_ROOT / "patches" / "meta" / "patches_all_enriched.csv"

MIN_CANOPY_FRAC = 0.25
BATCH_SAVE = 100   # autosave tiap 100 patch

def load_rgb_patch_u8(p):
    img = cv2.imread(str(p), cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(p)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_th_patch(p):
    arr = np.load(str(p)).astype(np.float32)
    if arr.ndim != 2:
        raise ValueError(arr.shape)
    return arr

def canopy_mask_v2_from_rgb(rgb_u8):
    rgb = rgb_u8.astype(np.float32)
    r, g, b = rgb[...,0], rgb[...,1], rgb[...,2]
    exg = 2*g - r - b

    lo, hi = np.percentile(exg, 2), np.percentile(exg, 98)
    exg_norm = np.clip((exg - lo) / (hi - lo + 1e-6), 0, 1)

    hsv = cv2.cvtColor(rgb_u8, cv2.COLOR_RGB2HSV)
    H,S,V = hsv[...,0], hsv[...,1], hsv[...,2]
    green = (H>=25)&(H<=95)&(S>=30)&(V>=30)

    mask = green & (exg_norm > 0.35)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    mask = cv2.morphologyEx(mask.astype(np.uint8)*255, cv2.MORPH_OPEN, k, 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, 2)

    return mask > 0


# ---- LOAD PATCH META ----
patches = pd.read_csv(PATCHES_ALL_CSV)
patches["patch_id"] = patches["patch_id"].astype(str)
print("Total patches:", len(patches))

# ---- RESUME LOGIC ----
if THERM_FEAT_CSV.exists():
    done = pd.read_csv(THERM_FEAT_CSV)
    done_ids = set(done["patch_id"].astype(str).tolist())
    print("Resume mode: already processed:", len(done_ids))
else:
    done = pd.DataFrame()
    done_ids = set()
    print("Fresh run")

todo = patches[~patches["patch_id"].isin(done_ids)].reset_index(drop=True)
print("Remaining to process:", len(todo))

rows = []
bad = 0

# ---- MAIN LOOP ----
for i, rr in enumerate(tqdm(todo.itertuples(), total=len(todo))):
    pid = rr.patch_id
    try:
        rgb = load_rgb_patch_u8(rr.rgb_patch_path)
        th  = load_th_patch(rr.thermal_patch_path)

        mask = canopy_mask_v2_from_rgb(rgb)
        canopy_frac = float(mask.mean())

        th_can = th.copy()
        th_can[~mask] = np.nan

        if np.isfinite(np.nanmean(th_can)):
            tc_mean = float(np.nanmean(th_can))
            tc_p90  = float(np.nanpercentile(th_can, 90))
            tc_p95  = float(np.nanpercentile(th_can, 95))
        else:
            tc_mean = tc_p90 = tc_p95 = np.nan

        rows.append({
            "patch_id": pid,
            "canopy_frac": canopy_frac,
            "tc_mean_canopy": tc_mean,
            "tc_p90_canopy": tc_p90,
            "tc_p95_canopy": tc_p95,
        })

    except Exception as e:
        bad += 1
        continue

    # ---- AUTOSAVE ----
    if len(rows) >= BATCH_SAVE:
        df_new = pd.DataFrame(rows)
        df_all = pd.concat([done, df_new], ignore_index=True)
        df_all.to_csv(THERM_FEAT_CSV, index=False)
        done = df_all
        rows = []
        print(f"💾 autosave @ {len(done)} rows")

# ---- FINAL SAVE (sisa) ----
if rows:
    df_new = pd.DataFrame(rows)
    df_all = pd.concat([done, df_new], ignore_index=True)
    df_all.to_csv(THERM_FEAT_CSV, index=False)
    done = df_all

print("✅ Saved thermal features:", THERM_FEAT_CSV)
print("Rows:", len(done), "| Bad:", bad)

# ---- QC + ENRICH ----
done["qc_canopy_ok"] = done["canopy_frac"] >= MIN_CANOPY_FRAC

enriched = patches.merge(
    done[["patch_id","canopy_frac","qc_canopy_ok"]],
    on="patch_id", how="left"
)
enriched["keep_for_model"] = enriched["qc_canopy_ok"].fillna(False)

enriched.to_csv(PATCHES_ENRICHED_CSV, index=False)

print("✅ Saved enriched patches:", PATCHES_ENRICHED_CSV)
print(enriched["keep_for_model"].value_counts())


In [ ]:
# ============================================
# EVIDENCE CELL — Canopy Masking Proof (Patch-level)
#   Show: RGB, Mask, Thermal raw, Thermal masked, Overlay + stats
#   Optional: save PNG evidence to Drive
# ============================================

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from pathlib import Path

# --------- CONFIG ----------
PATCHES_ALL_CSV = FIX_ROOT / "patches" / "meta" / "patches_all.csv"
EVID_DIR = FIX_ROOT / "reports" / "evidence_canopy_mask"
ensure_dir(EVID_DIR)

N_SHOW = 6                 # tampilkan berapa contoh
SAVE_PNG = True            # True = simpan ke Drive
SEED_LOCAL = 42

# Kalau kamu mau pilih patch_id spesifik, isi list ini (kalau kosong -> random)
SELECT_PATCH_IDS = []  # contoh: ["DJI_..._r000_c000", "DJI_..._r001_c003"]

# --------- FUNCTIONS ----------
def load_rgb_patch_u8(p: str) -> np.ndarray:
    img = cv2.imread(str(p), cv2.IMREAD_COLOR)
    if img is None:
        raise FileNotFoundError(p)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_th_patch(p: str) -> np.ndarray:
    arr = np.load(str(p)).astype(np.float32)
    if arr.ndim != 2:
        raise ValueError(f"thermal patch not 2D: {arr.shape}")
    return arr

def canopy_mask_v2_from_rgb(rgb_u8: np.ndarray) -> np.ndarray:
    rgb = rgb_u8.astype(np.float32)
    r, g, b = rgb[...,0], rgb[...,1], rgb[...,2]
    exg = 2*g - r - b

    lo = np.percentile(exg, 2)
    hi = np.percentile(exg, 98)
    exg_norm = (exg - lo) / (hi - lo + 1e-6)
    exg_norm = np.clip(exg_norm, 0, 1)

    hsv = cv2.cvtColor(rgb_u8, cv2.COLOR_RGB2HSV)
    H = hsv[...,0]; S = hsv[...,1]; V = hsv[...,2]
    green_hsv = (H >= 25) & (H <= 95) & (S >= 30) & (V >= 30)
    green_exg = exg_norm > 0.35

    mask = green_hsv & green_exg

    # morph clean
    mask_u8 = (mask.astype(np.uint8) * 255)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask_u8 = cv2.morphologyEx(mask_u8, cv2.MORPH_OPEN, k, iterations=1)
    mask_u8 = cv2.morphologyEx(mask_u8, cv2.MORPH_CLOSE, k, iterations=2)
    return mask_u8 > 0

def compute_thermal_stats(th: np.ndarray, mask: np.ndarray):
    th_can = th.copy()
    th_can[~mask] = np.nan
    canopy_frac = float(mask.mean())
    if np.isfinite(np.nanmean(th_can)):
        tc_mean = float(np.nanmean(th_can))
        tc_p90  = float(np.nanpercentile(th_can, 90))
        tc_p95  = float(np.nanpercentile(th_can, 95))
    else:
        tc_mean = tc_p90 = tc_p95 = np.nan
    return canopy_frac, tc_mean, tc_p90, tc_p95, th_can

def overlay_mask_on_rgb(rgb: np.ndarray, mask: np.ndarray, alpha=0.35):
    # overlay hijau pada area mask
    overlay = rgb.copy().astype(np.float32)
    green = np.zeros_like(overlay); green[...,1] = 255
    overlay[mask] = (1-alpha)*overlay[mask] + alpha*green[mask]
    return overlay.astype(np.uint8)

# --------- LOAD META ----------
df = pd.read_csv(PATCHES_ALL_CSV)
df["patch_id"] = df["patch_id"].astype(str)

if SELECT_PATCH_IDS:
    show_df = df[df["patch_id"].isin(SELECT_PATCH_IDS)].copy()
    if len(show_df) == 0:
        raise ValueError("SELECT_PATCH_IDS tidak ditemukan di patches_all.csv")
else:
    show_df = df.sample(n=min(N_SHOW, len(df)), random_state=SEED_LOCAL).copy()

show_df = show_df.reset_index(drop=True)

print("Showing:", len(show_df), "patches")
print("Evidence save dir:", EVID_DIR)

# --------- VISUALIZE ----------
for i, rr in show_df.iterrows():
    pid = rr["patch_id"]
    rgbp = rr["rgb_patch_path"]
    thp  = rr["thermal_patch_path"]

    rgb = load_rgb_patch_u8(rgbp)
    th  = load_th_patch(thp)
    mask = canopy_mask_v2_from_rgb(rgb)
    canopy_frac, tc_mean, tc_p90, tc_p95, th_can = compute_thermal_stats(th, mask)
    rgb_ov = overlay_mask_on_rgb(rgb, mask, alpha=0.35)

    # plot
    fig = plt.figure(figsize=(14, 8))
    fig.suptitle(f"{pid} | canopy_frac={canopy_frac:.3f} | Tc_mean={tc_mean:.2f} | p90={tc_p90:.2f} | p95={tc_p95:.2f}", fontsize=12)

    ax1 = plt.subplot(2,3,1); ax1.imshow(rgb); ax1.set_title("RGB patch"); ax1.axis("off")
    ax2 = plt.subplot(2,3,2); ax2.imshow(mask, cmap="gray"); ax2.set_title("Canopy mask"); ax2.axis("off")
    ax3 = plt.subplot(2,3,3); ax3.imshow(rgb_ov); ax3.set_title("RGB + mask overlay"); ax3.axis("off")

    ax4 = plt.subplot(2,3,4); ax4.imshow(th, cmap="inferno"); ax4.set_title("Thermal raw"); ax4.axis("off")
    ax5 = plt.subplot(2,3,5); ax5.imshow(th_can, cmap="inferno"); ax5.set_title("Thermal masked (canopy only)"); ax5.axis("off")
    ax6 = plt.subplot(2,3,6); ax6.hist(th[np.isfinite(th)].ravel(), bins=30); ax6.set_title("Thermal raw histogram")

    plt.tight_layout()
    plt.show()

    # save evidence PNG (optional)
    if SAVE_PNG:
        out_png = EVID_DIR / f"{pid}_evidence.png"
        fig.savefig(out_png, dpi=200, bbox_inches="tight")
        plt.close(fig)
        print("✅ Saved:", out_png)

print("DONE. Evidence ready for report.")


## 3.4 Auto-labeling + Merge Rules

In [ ]:
# ============================================
# CHECK GOLD LABEL SUMMARY
#   - total labeled rows
#   - normal / stress / skip / empty
#   - optional: cek coverage terhadap template (jika ada)
# ============================================

import pandas as pd
import numpy as np

GOLD_CSV = FIX_ROOT / "labels" / "gold" / "labels_gold.csv"
TEMPLATE_CSV = FIX_ROOT / "labels" / "gold" / "labels_gold_template.csv"  # optional

if not GOLD_CSV.exists():
    raise FileNotFoundError(f"Tidak ditemukan: {GOLD_CSV}")

gold = pd.read_csv(GOLD_CSV)
print("✅ Loaded:", GOLD_CSV)
print("Rows:", len(gold))
print("Columns:", list(gold.columns))

# pastikan patch_id string
if "patch_id" in gold.columns:
    gold["patch_id"] = gold["patch_id"].astype(str)

# label_manual bisa kosong / string / angka -> normalize
if "label_manual" not in gold.columns:
    raise ValueError("Kolom 'label_manual' tidak ada di labels_gold.csv")

gold["label_manual_num"] = pd.to_numeric(gold["label_manual"], errors="coerce")

# definisi skip:
# - label_manual kosong/NaN -> dianggap skip/empty
# - atau notes mengandung "[SKIP]"
notes = gold["notes"].fillna("").astype(str) if "notes" in gold.columns else pd.Series([""]*len(gold))
is_skip_note = notes.str.contains(r"\[SKIP\]", case=False, regex=True)

is_empty_label = gold["label_manual_num"].isna()
is_stress = gold["label_manual_num"] == 1
is_normal = gold["label_manual_num"] == 0

# hitung
n_total = len(gold)
n_stress = int(is_stress.sum())
n_normal = int(is_normal.sum())
n_empty = int(is_empty_label.sum())

# jika kamu pakai tombol skip, biasanya label_manual kosong + notes [SKIP]
n_skip = int((is_empty_label | is_skip_note).sum())
n_empty_no_skipflag = int((is_empty_label & ~is_skip_note).sum())

# unik patch
n_unique = gold["patch_id"].nunique() if "patch_id" in gold.columns else np.nan
n_dup = int(n_total - n_unique) if n_unique is not np.nan else 0

print("\n--- GOLD LABEL SUMMARY ---")
print("Total rows (saved):", n_total)
print("Unique patch_id    :", n_unique)
print("Duplicates         :", n_dup)

print("\nLabel counts:")
print("  Stress (1)       :", n_stress)
print("  Normal (0)       :", n_normal)
print("  Empty label      :", n_empty)
print("  Skip (empty OR [SKIP]) :", n_skip)
print("  Empty but no [SKIP]    :", n_empty_no_skipflag)

# sanity: hanya label valid (0/1)
valid = gold[gold["label_manual_num"].isin([0,1])].copy()
print("\nValid labeled (0/1):", len(valid))
if len(valid) > 0:
    print("Valid distribution:")
    print(valid["label_manual_num"].value_counts().rename({0:"Normal(0)",1:"Stress(1)"}))

# optional: coverage vs template
if TEMPLATE_CSV.exists() and "patch_id" in gold.columns:
    tpl = pd.read_csv(TEMPLATE_CSV)
    tpl["patch_id"] = tpl["patch_id"].astype(str)

    tpl_ids = set(tpl["patch_id"].tolist())
    gold_ids = set(gold["patch_id"].tolist())

    in_tpl = len(gold_ids & tpl_ids)
    out_tpl = len(gold_ids - tpl_ids)

    print("\n--- COVERAGE VS TEMPLATE ---")
    print("Template rows:", len(tpl))
    print("Gold patch_id in template:", in_tpl)
    print("Gold patch_id NOT in template:", out_tpl)

    # progress relabel 1..700 (jika template memang 2000 pool)
    first700 = set(tpl["patch_id"].iloc[:700].tolist())
    labeled_first700 = len(gold_ids & first700)
    print("Labeled among template #1..#700:", labeled_first700, "/ 700")
else:
    print("\n(Info) Template tidak ditemukan, skip coverage check:", TEMPLATE_CSV)


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf

# ---------- PATHS (sesuaikan kalau strukturmu beda) ----------
ENRICHED_CSV   = FIX_ROOT / "patches" / "meta" / "patches_all_enriched.csv"         # dari 3.3
THERM_FEAT_CSV = FIX_ROOT / "features" / "thermal" / "thermal_features.csv"        # dari 3.3
GOLD_CSV       = FIX_ROOT / "labels" / "gold" / "labels_gold.csv"                  # dari 3.2
SILVER_DIR     = FIX_ROOT / "labels" / "silver"
SILVER_AUTO_CSV= SILVER_DIR / "labels_silver_auto.csv"                             # output autolabel confident (akan dibuat jika belum ada)
BASEMODEL_PATH = SILVER_DIR / "rgb_baseline_for_autolabel.keras"                   # model baseline RGB (akan dibuat jika belum ada)

FINAL_DIR      = FIX_ROOT / "labels" / "final"
FINAL_DIR.mkdir(parents=True, exist_ok=True)
OUT_FINAL_CSV  = FINAL_DIR / "patches_all_final_labeled.csv"

# ---------- PARAMS ----------
USE_KEEP_FOR_MODEL = True     # Opsi A: hanya patch padi
IMG_SIZE = 128
BATCH = 64
EPOCHS = 6
SEED = 42

# Auto-confident thresholds (ketat)
THR_POS = 0.95
THR_NEG = 0.05

# Fallback thermal-relative threshold (°C) untuk semua sisanya
DELTA_C = 0.5

tf.random.set_seed(SEED)
np.random.seed(SEED)

def ensure_exists(p: Path, name="file"):
    if not p.exists():
        raise FileNotFoundError(f"{name} tidak ditemukan: {p}")

def make_ds_rgb(paths, labels=None, training=False):
    def _load_rgb(p, y=None):
        img = tf.io.read_file(p)
        # patch RGB kamu PNG hasil extract; kalau JPG juga aman decode_jpeg
        img = tf.image.decode_image(img, channels=3, expand_animations=False)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = tf.cast(img, tf.float32) / 255.0
        if y is None:
            return img
        return img, tf.cast(y, tf.float32)

    if labels is None:
        ds = tf.data.Dataset.from_tensor_slices(paths)
        ds = ds.map(lambda p: _load_rgb(p, None), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        ds = tf.data.Dataset.from_tensor_slices((paths, labels))
        ds = ds.map(lambda p, y: _load_rgb(p, y), num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)

    return ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)

def build_rgb_baseline():
    base = tf.keras.applications.MobileNetV2(
        include_top=False, weights="imagenet",
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base.trainable = False
    inp = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inp*255.0)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="roc_auc"), tf.keras.metrics.AUC(curve="PR", name="pr_auc")]
    )
    return model

# ---------- LOAD REQUIRED ----------
ensure_exists(ENRICHED_CSV, "ENRICHED_CSV")
ensure_exists(THERM_FEAT_CSV, "THERM_FEAT_CSV")
ensure_exists(GOLD_CSV, "GOLD_CSV")

df = pd.read_csv(ENRICHED_CSV)
df["patch_id"] = df["patch_id"].astype(str)

feat = pd.read_csv(THERM_FEAT_CSV)
feat["patch_id"] = feat["patch_id"].astype(str)

gold = pd.read_csv(GOLD_CSV)
gold["patch_id"] = gold["patch_id"].astype(str)
gold["label_manual_num"] = pd.to_numeric(gold["label_manual"], errors="coerce")  # 0/1 atau NaN

# ---------- WORK PATCHES (Opsi A) ----------
if USE_KEEP_FOR_MODEL and "keep_for_model" in df.columns:
    work = df[df["keep_for_model"] == True].copy()
else:
    work = df.copy()

print("All patches:", len(df), "| Work patches (padi):", len(work))

# merge manual
work = work.merge(gold[["patch_id","label_manual_num"]], on="patch_id", how="left")
manual_valid = work["label_manual_num"].isin([0,1]).sum()

print("Manual labeled in work (valid 0/1):", int(manual_valid))

# ---------- TRAIN / LOAD RGB BASELINE FOR AUTO-LABEL ----------
# train only from manual valid (0/1)
train_gold = work[work["label_manual_num"].isin([0,1])].copy()
train_gold["y"] = train_gold["label_manual_num"].astype(int)

print("\n--- GOLD (work) distribution ---")
print(train_gold["y"].value_counts())

if len(train_gold) < 200:
    print("⚠️ WARNING: GOLD valid < 200, autolabel bisa kurang stabil.")

# load model if exists else train
if BASEMODEL_PATH.exists():
    model = tf.keras.models.load_model(BASEMODEL_PATH)
    print("\n✅ Loaded baseline model:", BASEMODEL_PATH)
else:
    print("\nTraining baseline RGB model for autolabel...")
    paths = train_gold["rgb_patch_path"].astype(str).tolist()
    y = train_gold["y"].astype(int).tolist()

    n = len(paths)
    idxs = np.arange(n)
    np.random.shuffle(idxs)
    cut = max(1, int(0.85*n))
    tr_idx, va_idx = idxs[:cut], idxs[cut:]

    ds_tr = make_ds_rgb([paths[i] for i in tr_idx], [y[i] for i in tr_idx], training=True)
    ds_va = make_ds_rgb([paths[i] for i in va_idx], [y[i] for i in va_idx], training=False)

    model = build_rgb_baseline()
    model.fit(ds_tr, validation_data=ds_va, epochs=EPOCHS, verbose=1)

    SILVER_DIR.mkdir(parents=True, exist_ok=True)
    model.save(BASEMODEL_PATH)
    print("✅ Saved baseline model:", BASEMODEL_PATH)

# ---------- PREDICT ALL WORK PATCHES ----------
all_paths = work["rgb_patch_path"].astype(str).tolist()
ds_all = make_ds_rgb(all_paths, labels=None, training=False)
probs = model.predict(ds_all, verbose=1).reshape(-1)

work["p_auto_rgb"] = probs
work["label_auto"] = np.nan
work.loc[work["p_auto_rgb"] >= THR_POS, "label_auto"] = 1
work.loc[work["p_auto_rgb"] <= THR_NEG, "label_auto"] = 0
work["auto_confident"] = work["label_auto"].notna()

# save SILVER confident only (optional artifact)
silver = work[work["auto_confident"]][["patch_id","label_auto","p_auto_rgb"]].copy()
silver.to_csv(SILVER_AUTO_CSV, index=False)
print("✅ Saved silver auto confident:", SILVER_AUTO_CSV, "| rows:", len(silver))

# ---------- THERMAL-RELATIVE FALLBACK ----------
# join thermal feature (tc_mean_canopy) + compute tc_rel per frame
work = work.merge(feat[["patch_id","tc_mean_canopy"]], on="patch_id", how="left")

# safety: frame_id must exist
if "frame_id" not in work.columns:
    raise ValueError("Kolom frame_id tidak ditemukan di enriched meta. Pastikan patches_all.csv/enriched menyimpan frame_id.")

work["tc_med_frame"] = work.groupby("frame_id")["tc_mean_canopy"].transform("median")
work["tc_rel"] = work["tc_mean_canopy"] - work["tc_med_frame"]

# ---------- MERGE LABELS: manual > auto > fallback ----------
work["y_stage1"] = work["label_manual_num"]
mask_stage1_nan = work["y_stage1"].isna()
work.loc[mask_stage1_nan, "y_stage1"] = work.loc[mask_stage1_nan, "label_auto"]

# fallback for remaining NaN
mask_need_fallback = work["y_stage1"].isna()

# fallback rule: tc_rel >= DELTA_C => Stress else Normal
# if tc_rel NaN (should be rare) => default Normal (0) to avoid injecting stress noise
fallback = (work.loc[mask_need_fallback, "tc_rel"] >= DELTA_C).astype(float)
fallback = fallback.fillna(0.0)

work.loc[mask_need_fallback, "y_fallback"] = fallback
work["y_final"] = work["y_stage1"]
work.loc[work["y_final"].isna(), "y_final"] = work.loc[work["y_final"].isna(), "y_fallback"]

# label source
work["label_source"] = np.where(work["label_manual_num"].isin([0,1]), "manual",
                        np.where(work["label_auto"].notna(), "auto_confident", "fallback_thermal"))

# ---------- FINAL CHECKS ----------
n_total = len(work)
n_nan = int(pd.isna(work["y_final"]).sum())
print("\n--- FINAL LABEL SUMMARY (Work patches only / Opsi A) ---")
print("Total work patches:", n_total)
print("NaN y_final:", n_nan, "(harus 0)")

print("\nLabel distribution (y_final):")
print(work["y_final"].value_counts(dropna=False))

print("\nLabel source counts:")
print(work["label_source"].value_counts())

# ---------- SAVE FINAL DATASET ----------
work.to_csv(OUT_FINAL_CSV, index=False)
print("\n✅ Saved FINAL labeled dataset:", OUT_FINAL_CSV)
print("Columns include: patch_id, frame_id, rgb_patch_path, thermal_patch_path, keep_for_model, canopy_frac, tc_mean_canopy, tc_rel, p_auto_rgb, y_final, label_source")


# TAHAP 4 — Model Development

## 4.1 Dataset Builder + Splits (GroupKFold by frame_id)

In [ ]:
# ============================================
# TAHAP 4.1 — Dataset Builder + Splits (GroupKFold)
# - Load master final labeled dataset
# - Build df_train
# - Create folds with GroupKFold (group = frame_id)
# - Save fold assignment to CSV (reproducible)
# - Build tf.data dual-input loader:
#     X_rgb: (128,128,3)
#     X_th : (128,128,1) thermal masked+normalized (mask from RGB patch)
#     y    : 0/1
# - Print sanity checks (counts, imbalance, leakage)
# ============================================

import os, math
import numpy as np
import pandas as pd
from pathlib import Path
import cv2
import tensorflow as tf
from sklearn.model_selection import GroupKFold

# ---------- PATHS ----------
MASTER_FINAL = FIX_ROOT / "labels" / "final" / "patches_all_final_labeled.csv"  # master (debug-friendly)
assert MASTER_FINAL.exists(), f"Not found: {MASTER_FINAL}"

FOLD_DIR = FIX_ROOT / "models" / "folds"
FOLD_DIR.mkdir(parents=True, exist_ok=True)
FOLDS_CSV = FOLD_DIR / "patch_folds.csv"

# ---------- PARAMS ----------
IMG_SIZE = 128
BATCH = 64
N_SPLITS = 5
SEED = 42

tf.random.set_seed(SEED)
np.random.seed(SEED)

# ---------- LOAD ----------
df = pd.read_csv(MASTER_FINAL)
print("✅ Loaded:", MASTER_FINAL)
print("Rows:", len(df), "| Unique frames:", df["frame_id"].nunique())

need = ["patch_id","frame_id","rgb_patch_path","thermal_patch_path","y_final"]
miss = [c for c in need if c not in df.columns]
if miss:
    raise ValueError(f"Kolom wajib hilang di master final: {miss}")

# Pastikan y_final 0/1
df["y_final"] = pd.to_numeric(df["y_final"], errors="coerce").astype("Int64")
if df["y_final"].isna().any():
    raise ValueError("Masih ada NaN di y_final. Master final harus full-labeled untuk Opsi A.")

# ---------- RECONSTRUCT label_source (kalau tidak ada) ----------
if "label_source" not in df.columns:
    # logic: manual if label_manual_num valid; else auto_confident if label_auto exists; else fallback_thermal
    if "label_manual_num" in df.columns:
        manual_mask = df["label_manual_num"].isin([0,1])
    else:
        manual_mask = pd.Series(False, index=df.index)

    if "label_auto" in df.columns:
        auto_mask = df["label_auto"].notna()
    else:
        auto_mask = pd.Series(False, index=df.index)

    df["label_source"] = np.where(manual_mask, "manual",
                          np.where(auto_mask, "auto_confident", "fallback_thermal"))
    print("ℹ️ label_source tidak ada → dibuat ulang dari kolom yang tersedia.")

# ---------- BUILD df_train (untuk training model, kita pakai subset sesuai mode nanti) ----------
# di tahap 4.1 kita simpan fold assignment untuk semua patch kerja
df_train = df.copy()

# ---------- GROUPKFOLD ----------
groups = df_train["frame_id"].astype(str).values
gkf = GroupKFold(n_splits=N_SPLITS)

df_train["fold"] = -1
for fold_id, (_, val_idx) in enumerate(gkf.split(df_train, df_train["y_final"].values, groups=groups)):
    df_train.loc[val_idx, "fold"] = fold_id

assert (df_train["fold"] >= 0).all(), "Ada fold yang belum terisi."

# Save fold assignment (reproducible)
df_train.to_csv(FOLDS_CSV, index=False)
print("\n✅ Saved folds:", FOLDS_CSV)
print("Fold counts:")
display(df_train["fold"].value_counts().sort_index())

# ---------- SANITY CHECK: no frame overlap between train/val for each fold ----------
def check_leakage(df_in, fold_id):
    tr = df_in[df_in["fold"] != fold_id]
    va = df_in[df_in["fold"] == fold_id]
    tr_frames = set(tr["frame_id"].astype(str))
    va_frames = set(va["frame_id"].astype(str))
    inter = tr_frames.intersection(va_frames)
    return len(inter)

print("\nLeakage check (frame_id overlap):")
for f in range(N_SPLITS):
    inter = check_leakage(df_train, f)
    print(f"  Fold {f}: overlap frames =", inter)

# ---------- Label distribution per fold (y_final) ----------
print("\nLabel distribution per fold (y_final):")
for f in range(N_SPLITS):
    va = df_train[df_train["fold"] == f]
    counts = va["y_final"].value_counts().sort_index()
    n0 = int(counts.get(0,0)); n1 = int(counts.get(1,0))
    print(f"  Fold {f} | val size={len(va)} | Normal={n0} | Stress={n1}")

# ---------- Dual-input loader: RGB + Thermal (masked+normalized) ----------
def canopy_mask_v2_from_rgb(rgb_u8: np.ndarray) -> np.ndarray:
    # RGB uint8 (H,W,3)
    rgb = rgb_u8.astype(np.float32)
    r, g, b = rgb[...,0], rgb[...,1], rgb[...,2]
    exg = 2*g - r - b

    lo = np.percentile(exg, 2)
    hi = np.percentile(exg, 98)
    exg_norm = (exg - lo) / (hi - lo + 1e-6)
    exg_norm = np.clip(exg_norm, 0, 1)

    hsv = cv2.cvtColor(rgb_u8, cv2.COLOR_RGB2HSV)
    H = hsv[...,0]; S = hsv[...,1]; V = hsv[...,2]
    green_hsv = (H >= 25) & (H <= 95) & (S >= 30) & (V >= 30)
    green_exg = exg_norm > 0.35

    mask = green_hsv & green_exg

    mask_u8 = (mask.astype(np.uint8) * 255)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    mask_u8 = cv2.morphologyEx(mask_u8, cv2.MORPH_OPEN, k, iterations=1)
    mask_u8 = cv2.morphologyEx(mask_u8, cv2.MORPH_CLOSE, k, iterations=2)
    return (mask_u8 > 0)

def _load_one(rgb_path_b, th_path_b, y):
    rgb_path = rgb_path_b.decode("utf-8")
    th_path  = th_path_b.decode("utf-8")

    # RGB
    bgr = cv2.imread(rgb_path, cv2.IMREAD_COLOR)
    if bgr is None:
        raise FileNotFoundError(rgb_path)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    if rgb.shape[0] != IMG_SIZE or rgb.shape[1] != IMG_SIZE:
        rgb = cv2.resize(rgb, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
    rgb_f = rgb.astype(np.float32) / 255.0  # (128,128,3)

    # Thermal
    th = np.load(th_path).astype(np.float32)
    if th.ndim != 2:
        raise ValueError(f"Thermal patch not 2D: {th.shape}")
    if th.shape != (IMG_SIZE, IMG_SIZE):
        th = cv2.resize(th, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)

    # Mask canopy from RGB
    mask = canopy_mask_v2_from_rgb((rgb_f*255).astype(np.uint8))  # bool (128,128)
    th_can = th.copy()
    th_can[~mask] = np.nan

    # Normalize thermal within canopy (percentile 1-99), fallback to global if all NaN
    vals = th_can[np.isfinite(th_can)]
    if vals.size < 10:
        vals = th[np.isfinite(th)]
    p1 = np.percentile(vals, 1) if vals.size else 0.0
    p99= np.percentile(vals, 99) if vals.size else 1.0
    th_norm = (th - p1) / (p99 - p1 + 1e-6)
    th_norm = np.clip(th_norm, 0.0, 1.0).astype(np.float32)  # (128,128)
    th_norm = th_norm[..., None]  # (128,128,1)

    return rgb_f.astype(np.float32), th_norm, np.float32(y)

def tf_load(rgb_path, th_path, y):
    rgb, th, yy = tf.numpy_function(
        func=_load_one,
        inp=[rgb_path, th_path, y],
        Tout=[tf.float32, tf.float32, tf.float32]
    )
    rgb.set_shape((IMG_SIZE, IMG_SIZE, 3))
    th.set_shape((IMG_SIZE, IMG_SIZE, 1))
    yy.set_shape(())
    return (rgb, th), yy

def make_ds(df_part, training=False):
    rgb_paths = df_part["rgb_patch_path"].astype(str).values
    th_paths  = df_part["thermal_patch_path"].astype(str).values
    y         = df_part["y_final"].astype(int).values

    ds = tf.data.Dataset.from_tensor_slices((rgb_paths, th_paths, y))
    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(tf_load, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)
    return ds

print("\n✅ 4.1 DONE: folds ready + data loader functions defined.")


## 4.2 Model Definition (Late Fusion: RGB MobileNetV2 + Thermal CNN)

In [ ]:
# ============================================
# TAHAP 4.2 — Model Definition (Late Fusion)
# RGB: MobileNetV2 pretrained
# Thermal: small CNN (1-channel)
# Fusion: concat → dense → sigmoid
# Compile: ROC-AUC, PR-AUC, Recall, Precision
# ============================================

import tensorflow as tf

def build_late_fusion_model(img_size=128, lr=1e-4):
    # ---- RGB branch ----
    inp_rgb = tf.keras.Input((img_size, img_size, 3), name="rgb")
    base_rgb = tf.keras.applications.MobileNetV2(
        include_top=False, weights="imagenet",
        input_shape=(img_size, img_size, 3)
    )
    base_rgb.trainable = False  # start frozen; bisa fine-tune nanti
    x_rgb = tf.keras.applications.mobilenet_v2.preprocess_input(inp_rgb * 255.0)
    x_rgb = base_rgb(x_rgb, training=False)
    x_rgb = tf.keras.layers.GlobalAveragePooling2D(name="gap_rgb")(x_rgb)

    # ---- Thermal branch ----
    inp_th = tf.keras.Input((img_size, img_size, 1), name="th")
    x_th = tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu")(inp_th)
    x_th = tf.keras.layers.MaxPool2D()(x_th)
    x_th = tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu")(x_th)
    x_th = tf.keras.layers.MaxPool2D()(x_th)
    x_th = tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu")(x_th)
    x_th = tf.keras.layers.GlobalAveragePooling2D(name="gap_th")(x_th)

    # ---- Fusion ----
    x = tf.keras.layers.Concatenate(name="fusion")([x_rgb, x_th])
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid", name="p_stress")(x)

    model = tf.keras.Model(inputs=[inp_rgb, inp_th], outputs=out, name="late_fusion_rgb_th")

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
            tf.keras.metrics.Recall(name="recall"),
            tf.keras.metrics.Precision(name="precision"),
        ]
    )
    return model

model = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)
model.summary()

print("\nTotal params:", model.count_params())


## 4.3 Train Mode A

In [ ]:
# ============================================
# TAHAP 4.3 — Train Mode A (Scientific: GOLD-only)
# Train: label_source == manual
# Val  : pilih 1 fold (mis. fold=0)
# Save : model_gold.keras
# ============================================

import tensorflow as tf
import numpy as np
from pathlib import Path

MODEL_DIR = FIX_ROOT / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)
OUT_GOLD = MODEL_DIR / "model_gold.keras"

VAL_FOLD = 0

# Filter GOLD-only
df_gold = df_train[df_train["label_source"] == "manual"].copy()
print("GOLD-only rows:", len(df_gold), "| frames:", df_gold["frame_id"].nunique())
print("Label dist (GOLD):")
display(df_gold["y_final"].value_counts())

# Split by fold
tr = df_gold[df_gold["fold"] != VAL_FOLD].copy()
va = df_gold[df_gold["fold"] == VAL_FOLD].copy()

print(f"\nFold {VAL_FOLD} splits (GOLD-only):")
print("Train:", len(tr), "| Val:", len(va))
print("Train label dist:")
display(tr["y_final"].value_counts())
print("Val label dist:")
display(va["y_final"].value_counts())

# Leakage check
inter = set(tr["frame_id"]).intersection(set(va["frame_id"]))
print("Frame overlap:", len(inter))

ds_tr = make_ds(tr, training=True)
ds_va = make_ds(va, training=False)

model_gold = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)

cb = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_pr_auc", mode="max",
        patience=3, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(OUT_GOLD),
        monitor="val_pr_auc", mode="max",
        save_best_only=True, verbose=1
    )
]

hist = model_gold.fit(ds_tr, validation_data=ds_va, epochs=25, callbacks=cb, verbose=1)

# ringkas best epoch
best = np.argmax(hist.history["val_pr_auc"]) + 1
best_val = np.max(hist.history["val_pr_auc"])
print(f"\n✅ Best epoch (by val_pr_auc): {best} | val_pr_auc={best_val:.4f}")
print("✅ Saved best model:", OUT_GOLD)


In [ ]:
from sklearn.model_selection import GroupKFold
import numpy as np

N_SPLITS = 5
gkf = GroupKFold(n_splits=N_SPLITS)

df_gold = df_train[df_train["label_source"]=="manual"].copy()
groups = df_gold["frame_id"].astype(str).values
y = df_gold["y_final"].astype(int).values

fold_metrics = []

for f, (tr_idx, va_idx) in enumerate(gkf.split(df_gold, y, groups=groups)):
    tr = df_gold.iloc[tr_idx].copy()
    va = df_gold.iloc[va_idx].copy()

    ds_tr = make_ds(tr, training=True)
    ds_va = make_ds(va, training=False)

    model = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)
    model.fit(ds_tr, validation_data=ds_va, epochs=25, callbacks=cb, verbose=0)

    # ambil metrik val terakhir / atau evaluasi ulang
    res = model.evaluate(ds_va, verbose=0)
    fold_metrics.append(res)

# ringkas mean/std
fold_metrics = np.array(fold_metrics)
print("Mean metrics:", fold_metrics.mean(axis=0))
print("Std  metrics:", fold_metrics.std(axis=0))


## 4.4 Train Mode B

In [ ]:
# ============================================
# TAHAP 4.4 — Train Mode B (Deployment: GOLD + SILVER confident)
# Train: label_source in {manual, auto_confident}
# Val  : pilih 1 fold (mis. fold=0)
# Save : model_deploy.keras
# Print: data count per label_source
# ============================================

import tensorflow as tf
import numpy as np
from pathlib import Path

OUT_DEPLOY = MODEL_DIR / "model_deploy.keras"
VAL_FOLD = 0

df_deploy = df_train[df_train["label_source"].isin(["manual","auto_confident"])].copy()

print("Deploy-train rows:", len(df_deploy), "| frames:", df_deploy["frame_id"].nunique())
print("\nCounts per label_source:")
display(df_deploy["label_source"].value_counts())

print("\nLabel dist (deploy set):")
display(df_deploy["y_final"].value_counts())

tr = df_deploy[df_deploy["fold"] != VAL_FOLD].copy()
va = df_deploy[df_deploy["fold"] == VAL_FOLD].copy()

print(f"\nFold {VAL_FOLD} splits (deploy):")
print("Train:", len(tr), "| Val:", len(va))
print("Train label dist:")
display(tr["y_final"].value_counts())
print("Val label dist:")
display(va["y_final"].value_counts())

# Leakage check
inter = set(tr["frame_id"]).intersection(set(va["frame_id"]))
print("Frame overlap:", len(inter))

ds_tr = make_ds(tr, training=True)
ds_va = make_ds(va, training=False)

model_deploy = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)

cb = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_pr_auc", mode="max",
        patience=3, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(OUT_DEPLOY),
        monitor="val_pr_auc", mode="max",
        save_best_only=True, verbose=1
    )
]

hist = model_deploy.fit(ds_tr, validation_data=ds_va, epochs=25, callbacks=cb, verbose=1)

best = np.argmax(hist.history["val_pr_auc"]) + 1
best_val = np.max(hist.history["val_pr_auc"])
print(f"\n✅ Best epoch (by val_pr_auc): {best} | val_pr_auc={best_val:.4f}")
print("✅ Saved best model:", OUT_DEPLOY)


In [ ]:
df_deploy = df_train[df_train["label_source"].isin(["manual","auto_confident"])].copy()
groups = df_deploy["frame_id"].astype(str).values
y = df_deploy["y_final"].astype(int).values

gkf = GroupKFold(n_splits=5)
scores = []

for f, (tr_idx, va_idx) in enumerate(gkf.split(df_deploy, y, groups=groups)):
    tr = df_deploy.iloc[tr_idx].copy()
    va_all = df_deploy.iloc[va_idx].copy()

    # eval hanya manual di fold tsb
    va = va_all[va_all["label_source"]=="manual"].copy()
    if len(va) == 0:
        print("Fold", f, "skip: no manual in val")
        continue

    ds_tr = make_ds(tr, training=True)
    ds_va = make_ds(va, training=False)

    model = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)
    model.fit(ds_tr, validation_data=ds_va, epochs=25, callbacks=cb, verbose=0)

    res = model.evaluate(ds_va, verbose=0)
    scores.append(res)

scores = np.array(scores)
print("Mean metrics (eval manual-only):", scores.mean(axis=0))
print("Std  metrics:", scores.std(axis=0))


In [ ]:
import tensorflow as tf

candidates = []
for k, v in globals().items():
    if isinstance(v, tf.keras.Model):
        candidates.append(k)

print("Keras model objects in memory:")
for c in candidates:
    print("-", c)


In [ ]:
import tensorflow as tf

def inspect_model(name, mdl):
    print(f"\n=== {name} ===")
    print("Inputs:")
    for i, inp in enumerate(mdl.inputs):
        print(f"  input[{i}]: shape={inp.shape}")
    print("Outputs:", mdl.outputs[0].shape)

inspect_model("model", model)
inspect_model("model_final", model_final)
inspect_model("model_ft", model_ft)


# TAHAP 5 - Evaluation & Comparison

## 5.1 Setup Evaluasi

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix
)

THR = 0.5
N_SPLITS = df_train["fold"].nunique()

def compute_metrics(y_true, y_prob, thr=0.5):
    y_pred = (y_prob >= thr).astype(int)
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "cm": confusion_matrix(y_true, y_pred)
    }


## 5.2 Dataset Evaluasi

In [ ]:
df_gold = df_train[df_train["label_source"] == "manual"].copy()

print("GOLD rows:", len(df_gold))
print("Frames:", df_gold["frame_id"].nunique())
print("Label dist:")
display(df_gold["y_final"].value_counts())

print("\nLabel dist per fold:")
for f in sorted(df_gold["fold"].unique()):
    sub = df_gold[df_gold["fold"] == f]
    print(f"Fold {f}:")
    display(sub["y_final"].value_counts())


## 5.3 RGB-only Model + RGB-only Data Loader

### 5.3A Model RGB-only

In [ ]:
import tensorflow as tf

def build_rgb_only_model(img_size=128, lr=1e-4):
    inp = tf.keras.Input((img_size, img_size, 3), name="rgb")
    base = tf.keras.applications.MobileNetV2(
        include_top=False, weights="imagenet",
        input_shape=(img_size, img_size, 3)
    )
    base.trainable = False

    x = tf.keras.applications.mobilenet_v2.preprocess_input(inp * 255.0)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=inp, outputs=out, name="rgb_only")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc")
        ]
    )
    return model


### 5.3B RGB-only Data Loader

In [ ]:
def make_ds_rgb(df_part, training=False):
    rgb_paths = df_part["rgb_patch_path"].astype(str).values
    y = df_part["y_final"].astype(int).values

    def _load_rgb(rgb_path_b, y):
        rgb_path = rgb_path_b.decode("utf-8")
        bgr = cv2.imread(rgb_path, cv2.IMREAD_COLOR)
        if bgr is None:
            raise FileNotFoundError(rgb_path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        if rgb.shape[:2] != (IMG_SIZE, IMG_SIZE):
            rgb = cv2.resize(rgb, (IMG_SIZE, IMG_SIZE))
        rgb = rgb.astype(np.float32) / 255.0
        return rgb, np.float32(y)

    def tf_load(rgb_path, y):
        rgb, yy = tf.numpy_function(
            _load_rgb, [rgb_path, y], [tf.float32, tf.float32]
        )
        rgb.set_shape((IMG_SIZE, IMG_SIZE, 3))
        yy.set_shape(())
        return rgb, yy

    ds = tf.data.Dataset.from_tensor_slices((rgb_paths, y))
    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(tf_load, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)
    return ds


## 5.4 K-Fold Evaluation: RGB-only

In [ ]:
rgb_oof_rows = []
rgb_fold_metrics = []
rgb_cm_sum = np.zeros((2,2), dtype=int)

for f in range(N_SPLITS):
    tr = df_gold[df_gold["fold"] != f].copy()
    va = df_gold[df_gold["fold"] == f].copy()

    ds_tr = make_ds_rgb(tr, training=True)
    ds_va = make_ds_rgb(va, training=False)

    model = build_rgb_only_model(img_size=IMG_SIZE, lr=1e-4)
    model.fit(ds_tr, epochs=15, verbose=0)

    y_prob = model.predict(ds_va).ravel()
    y_true = va["y_final"].values

    m = compute_metrics(y_true, y_prob, THR)
    rgb_fold_metrics.append({k:v for k,v in m.items() if k!="cm"})
    rgb_cm_sum += m["cm"]

    for pid, fid, yt, yp in zip(va["patch_id"], va["frame_id"], y_true, y_prob):
        rgb_oof_rows.append({
            "patch_id": pid, "frame_id": fid, "fold": f,
            "y_true": yt, "y_prob": yp, "y_pred": int(yp>=THR)
        })

rgb_metrics_df = pd.DataFrame(rgb_fold_metrics)
display(rgb_metrics_df)
print("\nRGB-only mean ± std:")
display(rgb_metrics_df.agg(["mean","std"]))
print("RGB-only Confusion Matrix (aggregated):\n", rgb_cm_sum)

pd.DataFrame(rgb_oof_rows).to_csv("pred_rgb_oof.csv", index=False)


## 5.5 K-Fold Evaluation: Fusion

In [ ]:
fusion_oof_rows = []
fusion_fold_metrics = []
fusion_cm_sum = np.zeros((2,2), dtype=int)

for f in range(N_SPLITS):
    tr = df_gold[df_gold["fold"] != f].copy()
    va = df_gold[df_gold["fold"] == f].copy()

    ds_tr = make_ds(tr, training=True)
    ds_va = make_ds(va, training=False)

    model = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)
    model.fit(ds_tr, epochs=15, verbose=0)

    y_prob = model.predict(ds_va).ravel()
    y_true = va["y_final"].values

    m = compute_metrics(y_true, y_prob, THR)
    fusion_fold_metrics.append({k:v for k,v in m.items() if k!="cm"})
    fusion_cm_sum += m["cm"]

    for pid, fid, yt, yp in zip(va["patch_id"], va["frame_id"], y_true, y_prob):
        fusion_oof_rows.append({
            "patch_id": pid, "frame_id": fid, "fold": f,
            "y_true": yt, "y_prob": yp, "y_pred": int(yp>=THR)
        })

fusion_metrics_df = pd.DataFrame(fusion_fold_metrics)
display(fusion_metrics_df)
print("\nFusion mean ± std:")
display(fusion_metrics_df.agg(["mean","std"]))
print("Fusion Confusion Matrix (aggregated):\n", fusion_cm_sum)

pd.DataFrame(fusion_oof_rows).to_csv("pred_fusion_oof.csv", index=False)


## 5.6 Deploy Evaluation (YANG DIPAKAI YANG INI YA)


### 5.6.1 RGB ONLY

In [ ]:
import os, random
import numpy as np
import pandas as pd
from pathlib import Path

import tensorflow as tf
from sklearn.model_selection import GroupKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix
)

# ---------------------------
# 0) Reproducibility
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ---------------------------
# 1) PATHS (sesuaikan kalau FIX_ROOT kamu beda)
# ---------------------------
# Root project kamu (sesuai yang selama ini dipakai)
FIX_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")

# FINAL labeled dataset hasil autolabel config topk_10pct
FINAL_LABELED_CSV = FIX_ROOT / "labels" / "final" / "patches_all_final_labeled_topk_10pct.csv"

# Export output ke Drive
OUT_DIR = FIX_ROOT / "exports"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_OOF = OUT_DIR / "pred_rgb_oof_deployset_manual_autoconf_topk_10pct.csv"
OUT_MET = OUT_DIR / "metrics_rgb_deployset_manual_autoconf_topk_10pct.csv"

# ---------------------------
# 2) PARAMS
# ---------------------------
IMG_SIZE = 128
BATCH = 64
N_SPLITS = 5

EPOCHS = 30          # cukup besar, tapi akan dipotong EarlyStopping
LR_HEAD = 1e-4       # LR head (warm-up)
LR_FT   = 5e-5       # LR finetune
PATIENCE = 4

# ---------------------------
# 3) Load dataset FINAL (topk_10pct)
# ---------------------------
if not FINAL_LABELED_CSV.exists():
    raise FileNotFoundError(f"FINAL labeled CSV tidak ditemukan:\n{FINAL_LABELED_CSV}")

df = pd.read_csv(FINAL_LABELED_CSV)
print("Loaded:", FINAL_LABELED_CSV, "shape:", df.shape)
print("Columns:", list(df.columns))

# Pastikan kolom penting ada
need_cols = ["patch_id","frame_id","rgb_patch_path","y_final","label_source"]
for c in need_cols:
    if c not in df.columns:
        raise ValueError(f"Kolom wajib '{c}' tidak ada di CSV. Kolom tersedia: {list(df.columns)}")

# (Opsional tapi disarankan) filter hanya patch padi (kalau kolom ada)
for col in ["keep_for_model", "qc_canopy_ok"]:
    if col in df.columns:
        before = len(df)
        df = df[df[col] == True].copy()
        print(f"Filter {col}=True: {before} -> {len(df)}")

# Ambil hanya manual + auto_confident (target kamu = 2069)
df_eval = df[df["label_source"].isin(["manual","auto_confident"])].copy()

# pastikan label numeric 0/1
df_eval["y_final"] = pd.to_numeric(df_eval["y_final"], errors="coerce")
df_eval = df_eval[df_eval["y_final"].isin([0,1])].copy()
df_eval["y_final"] = df_eval["y_final"].astype(int)

df_eval = df_eval.reset_index(drop=True)
print("\nEVAL rows (manual+auto_confident):", len(df_eval))
print(df_eval["label_source"].value_counts())
print("Label dist:", df_eval["y_final"].value_counts().to_dict())

# ---------------------------
# 4) Buat fold by frame_id (anti leakage)
# ---------------------------
gkf = GroupKFold(n_splits=N_SPLITS)
df_eval["fold"] = -1
for fold, (_, va_idx) in enumerate(gkf.split(df_eval, df_eval["y_final"], groups=df_eval["frame_id"])):
    df_eval.loc[va_idx, "fold"] = fold

assert (df_eval["fold"] >= 0).all()
print("\nFold sizes:", df_eval["fold"].value_counts().sort_index().to_dict())

# ---------------------------
# 5) tf.data loader RGB
# ---------------------------
AUTOTUNE = tf.data.AUTOTUNE

def _load_rgb(path, y=None):
    # path: tf.string
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    if y is None:
        return img
    y = tf.cast(y, tf.float32)
    return img, y

def make_ds_rgb(df_part: pd.DataFrame, training: bool):
    paths = df_part["rgb_patch_path"].astype(str).values
    labels = df_part["y_final"].astype(int).values

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(lambda p, y: _load_rgb(p, y), num_parallel_calls=AUTOTUNE)

    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)

    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

# ---------------------------
# 6) Model RGB-only (MobileNetV2) + optional finetune
# ---------------------------
def build_rgb_only_model(lr=LR_HEAD, train_base=False):
    base = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base.trainable = bool(train_base)

    inp = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inp * 255.0)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
        ]
    )
    return model

# ---------------------------
# 7) Metrics helper + threshold search (maximize accuracy)
# ---------------------------
def best_thr_for_accuracy(y_true, y_prob):
    ths = np.linspace(0.05, 0.95, 181)
    best_t, best_acc = 0.5, -1
    for t in ths:
        acc = accuracy_score(y_true, (y_prob >= t).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_t = float(t)
    return best_t, best_acc

def compute_metrics(y_true, y_prob, thr):
    y_pred = (y_prob >= thr).astype(int)
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, zero_division=0)),
        "roc_auc": float(roc_auc_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
        "pr_auc": float(average_precision_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
        "cm": confusion_matrix(y_true, y_pred, labels=[0,1])
    }

# ---------------------------
# 8) K-Fold Training + OOF preds
# ---------------------------
rgb_oof_rows = []
fold_metrics = []
cm_sum = np.zeros((2,2), dtype=int)

for f in range(N_SPLITS):
    tr = df_eval[df_eval["fold"] != f].copy()
    va = df_eval[df_eval["fold"] == f].copy()

    ds_tr = make_ds_rgb(tr, training=True)
    ds_va = make_ds_rgb(va, training=False)

    # class_weight (lebih stabil)
    y_tr = tr["y_final"].values
    classes = np.array([0,1])
    cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
    class_weight = {0: float(cw[0]), 1: float(cw[1])}

    # --- stage A: train head (base frozen)
    model = build_rgb_only_model(lr=LR_HEAD, train_base=False)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
    ]

    model.fit(
        ds_tr,
        validation_data=ds_va,
        epochs=EPOCHS,
        verbose=0,
        class_weight=class_weight,
        callbacks=callbacks,
    )

    # --- stage B: light finetune (unfreeze last N layers of base)
    # (Ini sering ngangkat sedikit performa tanpa overfit terlalu cepat)
    base = None
    for lyr in model.layers:
        if isinstance(lyr, tf.keras.Model) and "mobilenetv2" in lyr.name.lower():
            base = lyr
            break
    # kalau tidak ketemu via loop di atas (struktur bisa beda), cari via model.get_layer
    if base is None:
        try:
            base = model.get_layer(index=1)
        except:
            base = None

    if base is not None:
        base.trainable = True
        # freeze awal, unfreeze bagian akhir saja
        # (angka 30 bisa kamu ubah: 20/30/40)
        for lyr in base.layers[:-30]:
            lyr.trainable = False

        model.compile(
            optimizer=tf.keras.optimizers.Adam(LR_FT),
            loss="binary_crossentropy",
            metrics=[tf.keras.metrics.AUC(name="roc_auc"),
                     tf.keras.metrics.AUC(curve="PR", name="pr_auc")]
        )

        model.fit(
            ds_tr,
            validation_data=ds_va,
            epochs=10,
            verbose=0,
            class_weight=class_weight,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)],
        )

    # Predict fold
    y_prob = model.predict(ds_va, verbose=0).ravel()
    y_true = va["y_final"].values.astype(int)

    thr_fold, acc_fold = best_thr_for_accuracy(y_true, y_prob)
    m = compute_metrics(y_true, y_prob, thr_fold)
    m_out = {k:v for k,v in m.items() if k != "cm"}
    m_out["thr_used"] = float(thr_fold)
    m_out["fold"] = int(f)
    m_out["n_val"] = int(len(va))
    fold_metrics.append(m_out)
    cm_sum += m["cm"]

    for pid, fid, yt, yp in zip(va["patch_id"].astype(str).values,
                                va["frame_id"].astype(str).values,
                                y_true, y_prob):
        rgb_oof_rows.append({
            "patch_id": pid,
            "frame_id": fid,
            "fold": int(f),
            "y_true": int(yt),
            "y_prob": float(yp),
            "thr_used": float(thr_fold),
            "y_pred": int(yp >= thr_fold),
            "label_source": "manual_or_autoconf"  # penanda, biar ga ketuker file
        })

# ---------------------------
# 9) Report + Save to Drive
# ---------------------------
met_df = pd.DataFrame(fold_metrics).sort_values("fold").reset_index(drop=True)
print("\nRGB-only (manual+auto_confident) per-fold:")
display(met_df)

print("\nMean ± Std:")
display(met_df.drop(columns=["fold"]).agg(["mean","std"]))

print("\nConfusion Matrix (aggregated):\n", cm_sum)

oof_df = pd.DataFrame(rgb_oof_rows)
oof_df.to_csv(OUT_OOF, index=False)
met_df.to_csv(OUT_MET, index=False)

print("\n✅ Saved OOF:", OUT_OOF)
print("✅ Saved metrics:", OUT_MET)
print("OOF shape:", oof_df.shape)


### 5.6.2 Thermal ONLY

In [ ]:
import os, random
import numpy as np
import pandas as pd
from pathlib import Path

import tensorflow as tf
from sklearn.model_selection import GroupKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix
)

# ---------------------------
# 0) Reproducibility
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ---------------------------
# 1) PATHS
# ---------------------------
FIX_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")
FINAL_LABELED_CSV = FIX_ROOT / "labels" / "final" / "patches_all_final_labeled_topk_10pct.csv"

OUT_DIR = FIX_ROOT / "exports"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_OOF = OUT_DIR / "pred_thermal_oof_deployset_manual_autoconf_topk_10pct.csv"
OUT_MET = OUT_DIR / "metrics_thermal_deployset_manual_autoconf_topk_10pct.csv"

# ---------------------------
# 2) PARAMS
# ---------------------------
IMG_SIZE = 128
BATCH = 64
N_SPLITS = 5

EPOCHS = 30
LR_HEAD = 1e-4
LR_FT = 5e-5
PATIENCE = 4

# ---------------------------
# 3) Load dataset FINAL
# ---------------------------
if not FINAL_LABELED_CSV.exists():
    raise FileNotFoundError(f"FINAL labeled CSV tidak ditemukan:\n{FINAL_LABELED_CSV}")

df = pd.read_csv(FINAL_LABELED_CSV)
print("Loaded:", FINAL_LABELED_CSV, "shape:", df.shape)

need_cols = ["patch_id","frame_id","thermal_patch_path","y_final","label_source"]
for c in need_cols:
    if c not in df.columns:
        raise ValueError(f"Kolom wajib '{c}' tidak ada. Kolom tersedia: {list(df.columns)}")

# opsional filter kualitas (kalau kolom ada)
for col in ["keep_for_model", "qc_canopy_ok"]:
    if col in df.columns:
        before = len(df)
        df = df[df[col] == True].copy()
        print(f"Filter {col}=True: {before} -> {len(df)}")

# hanya manual + auto_confident
df_eval = df[df["label_source"].isin(["manual","auto_confident"])].copy()
df_eval["y_final"] = pd.to_numeric(df_eval["y_final"], errors="coerce")
df_eval = df_eval[df_eval["y_final"].isin([0,1])].copy()
df_eval["y_final"] = df_eval["y_final"].astype(int)
df_eval = df_eval.reset_index(drop=True)

print("\nEVAL rows (manual+auto_confident):", len(df_eval))
print(df_eval["label_source"].value_counts())
print("Label dist:", df_eval["y_final"].value_counts().to_dict())

# pastikan file npy exist minimal sampling (biar cepat ketahuan path salah)
sample_paths = df_eval["thermal_patch_path"].astype(str).head(5).tolist()
for p in sample_paths:
    if not Path(p).exists():
        raise FileNotFoundError(f"Contoh thermal_patch_path tidak ditemukan:\n{p}")

# ---------------------------
# 4) Fold by frame_id
# ---------------------------
gkf = GroupKFold(n_splits=N_SPLITS)
df_eval["fold"] = -1
for fold, (_, va_idx) in enumerate(gkf.split(df_eval, df_eval["y_final"], groups=df_eval["frame_id"])):
    df_eval.loc[va_idx, "fold"] = fold
assert (df_eval["fold"] >= 0).all()
print("\nFold sizes:", df_eval["fold"].value_counts().sort_index().to_dict())

# ---------------------------
# 5) tf.data loader THERMAL (.npy)
# ---------------------------
AUTOTUNE = tf.data.AUTOTUNE

def _np_load_npy(path_bytes):
    # path_bytes: bytes
    p = path_bytes.decode("utf-8")
    arr = np.load(p).astype(np.float32)  # (H,W) = (128,128) atau sesuai patch
    return arr

def _load_th(path, y=None):
    # pakai numpy_function agar bisa np.load
    th = tf.numpy_function(_np_load_npy, [path], tf.float32)  # shape unknown
    th.set_shape((IMG_SIZE, IMG_SIZE))  # WAJIB biar graph stabil
    th = tf.expand_dims(th, axis=-1)    # (H,W,1)

    # normalisasi sederhana: per-patch z-score (lebih stabil daripada raw)
    mean = tf.reduce_mean(th)
    std = tf.math.reduce_std(th)
    th = (th - mean) / (std + 1e-6)

    if y is None:
        return th
    return th, tf.cast(y, tf.float32)

def make_ds_th(df_part: pd.DataFrame, training: bool):
    paths = df_part["thermal_patch_path"].astype(str).values
    labels = df_part["y_final"].astype(int).values

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(lambda p, y: _load_th(p, y), num_parallel_calls=AUTOTUNE)

    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)

    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

# ---------------------------
# 6) Model THERMAL-only (small CNN)
# ---------------------------
def build_thermal_only_model(lr=LR_HEAD):
    inp = tf.keras.Input((IMG_SIZE, IMG_SIZE, 1))
    x = inp

    x = tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
        ]
    )
    return model

# ---------------------------
# 7) Metrics helper + threshold search
# ---------------------------
def best_thr_for_accuracy(y_true, y_prob):
    ths = np.linspace(0.05, 0.95, 181)
    best_t, best_acc = 0.5, -1
    for t in ths:
        acc = accuracy_score(y_true, (y_prob >= t).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_t = float(t)
    return best_t, best_acc

def compute_metrics(y_true, y_prob, thr):
    y_pred = (y_prob >= thr).astype(int)
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, zero_division=0)),
        "roc_auc": float(roc_auc_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
        "pr_auc": float(average_precision_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
        "cm": confusion_matrix(y_true, y_pred, labels=[0,1])
    }

# ---------------------------
# 8) K-Fold Training + OOF
# ---------------------------
th_oof_rows = []
fold_metrics = []
cm_sum = np.zeros((2,2), dtype=int)

for f in range(N_SPLITS):
    tr = df_eval[df_eval["fold"] != f].copy()
    va = df_eval[df_eval["fold"] == f].copy()

    ds_tr = make_ds_th(tr, training=True)
    ds_va = make_ds_th(va, training=False)

    # class_weight
    y_tr = tr["y_final"].values
    classes = np.array([0,1])
    cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
    class_weight = {0: float(cw[0]), 1: float(cw[1])}

    model = build_thermal_only_model(lr=LR_HEAD)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
    ]

    model.fit(
        ds_tr,
        validation_data=ds_va,
        epochs=EPOCHS,
        verbose=0,
        class_weight=class_weight,
        callbacks=callbacks,
    )

    y_prob = model.predict(ds_va, verbose=0).ravel()
    y_true = va["y_final"].values.astype(int)

    thr_fold, _ = best_thr_for_accuracy(y_true, y_prob)

    m = compute_metrics(y_true, y_prob, thr_fold)
    m_out = {k:v for k,v in m.items() if k != "cm"}
    m_out["thr_used"] = float(thr_fold)
    m_out["fold"] = int(f)
    m_out["n_val"] = int(len(va))
    fold_metrics.append(m_out)
    cm_sum += m["cm"]

    for pid, fid, yt, yp in zip(va["patch_id"].astype(str).values,
                                va["frame_id"].astype(str).values,
                                y_true, y_prob):
        th_oof_rows.append({
            "patch_id": pid,
            "frame_id": fid,
            "fold": int(f),
            "y_true": int(yt),
            "y_prob": float(yp),
            "thr_used": float(thr_fold),
            "y_pred": int(yp >= thr_fold),
            "label_source": "manual_or_autoconf"
        })

# ---------------------------
# 9) Report + Save
# ---------------------------
met_df = pd.DataFrame(fold_metrics).sort_values("fold").reset_index(drop=True)
print("\nTHERMAL-only (manual+auto_confident) per-fold:")
display(met_df)

print("\nMean ± Std:")
display(met_df.drop(columns=["fold"]).agg(["mean","std"]))

print("\nConfusion Matrix (aggregated):\n", cm_sum)

oof_df = pd.DataFrame(th_oof_rows)
oof_df.to_csv(OUT_OOF, index=False)
met_df.to_csv(OUT_MET, index=False)

print("\n✅ Saved OOF:", OUT_OOF)
print("✅ Saved metrics:", OUT_MET)
print("OOF shape:", oof_df.shape)


### 5.6.3 FUSION (RGB + THERMAL)

In [ ]:
# ============================================
# FUSION (RGB+THERMAL) EVAL (manual + auto_confident) = dari FINAL thr_090_010
# - input: patches_all_final_labeled_thr_090_010.csv
# - output: metrics per-fold + OOF preds CSV (SAVE ke Drive)
# ============================================

import os, random
import numpy as np
import pandas as pd
from pathlib import Path

import tensorflow as tf
from sklearn.model_selection import GroupKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score, confusion_matrix
)

# ---------------------------
# 0) Reproducibility
# ---------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ---------------------------
# 1) PATHS
# ---------------------------
FIX_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")
FINAL_LABELED_CSV = FIX_ROOT / "labels" / "final" / "patches_all_final_labeled_topk_10pct.csv"

OUT_DIR = FIX_ROOT / "exports"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_OOF = OUT_DIR / "pred_fusion_oof_deployset_manual_autoconf_thr_topk_10pct.csv"
OUT_MET = OUT_DIR / "metrics_fusion_deployset_manual_autoconf_thr_topk_10pct.csv"

# ---------------------------
# 2) PARAMS
# ---------------------------
IMG_SIZE = 128
BATCH = 32          # fusion lebih berat, amanin RAM/VRAM
N_SPLITS = 5
EPOCHS = 30
LR = 1e-4
PATIENCE = 4

# ---------------------------
# 3) Load dataset FINAL
# ---------------------------
if not FINAL_LABELED_CSV.exists():
    raise FileNotFoundError(f"FINAL labeled CSV tidak ditemukan:\n{FINAL_LABELED_CSV}")

df = pd.read_csv(FINAL_LABELED_CSV)
print("Loaded:", FINAL_LABELED_CSV, "shape:", df.shape)

need_cols = ["patch_id","frame_id","rgb_patch_path","thermal_patch_path","y_final","label_source"]
for c in need_cols:
    if c not in df.columns:
        raise ValueError(f"Kolom wajib '{c}' tidak ada. Kolom tersedia: {list(df.columns)}")

# opsional filter kualitas (kalau kolom ada)
for col in ["keep_for_model", "qc_canopy_ok"]:
    if col in df.columns:
        before = len(df)
        df = df[df[col] == True].copy()
        print(f"Filter {col}=True: {before} -> {len(df)}")

# hanya manual + auto_confident
df_eval = df[df["label_source"].isin(["manual","auto_confident"])].copy()
df_eval["y_final"] = pd.to_numeric(df_eval["y_final"], errors="coerce")
df_eval = df_eval[df_eval["y_final"].isin([0,1])].copy()
df_eval["y_final"] = df_eval["y_final"].astype(int)
df_eval = df_eval.reset_index(drop=True)

print("\nEVAL rows (manual+auto_confident):", len(df_eval))
print(df_eval["label_source"].value_counts())
print("Label dist:", df_eval["y_final"].value_counts().to_dict())

# sanity check path sample
for p in df_eval["rgb_patch_path"].astype(str).head(3).tolist():
    if not Path(p).exists():
        raise FileNotFoundError(f"Contoh rgb_patch_path tidak ditemukan:\n{p}")
for p in df_eval["thermal_patch_path"].astype(str).head(3).tolist():
    if not Path(p).exists():
        raise FileNotFoundError(f"Contoh thermal_patch_path tidak ditemukan:\n{p}")

# ---------------------------
# 4) Fold by frame_id
# ---------------------------
gkf = GroupKFold(n_splits=N_SPLITS)
df_eval["fold"] = -1
for fold, (_, va_idx) in enumerate(gkf.split(df_eval, df_eval["y_final"], groups=df_eval["frame_id"])):
    df_eval.loc[va_idx, "fold"] = fold
assert (df_eval["fold"] >= 0).all()
print("\nFold sizes:", df_eval["fold"].value_counts().sort_index().to_dict())

# ---------------------------
# 5) tf.data loaders
# ---------------------------
AUTOTUNE = tf.data.AUTOTUNE

def _np_load_npy(path_bytes):
    p = path_bytes.decode("utf-8")
    arr = np.load(p).astype(np.float32)
    return arr

def _load_rgb(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

def _load_th(path):
    th = tf.numpy_function(_np_load_npy, [path], tf.float32)
    th.set_shape((IMG_SIZE, IMG_SIZE))
    th = tf.expand_dims(th, axis=-1)
    # per-patch z-score
    mean = tf.reduce_mean(th)
    std = tf.math.reduce_std(th)
    th = (th - mean) / (std + 1e-6)
    return th

def _load_fusion(rgb_path, th_path, y):
    rgb = _load_rgb(rgb_path)
    th  = _load_th(th_path)
    return (rgb, th), tf.cast(y, tf.float32)

def make_ds_fusion(df_part: pd.DataFrame, training: bool):
    rgb_paths = df_part["rgb_patch_path"].astype(str).values
    th_paths  = df_part["thermal_patch_path"].astype(str).values
    labels    = df_part["y_final"].astype(int).values

    ds = tf.data.Dataset.from_tensor_slices((rgb_paths, th_paths, labels))
    ds = ds.map(lambda rp, tp, y: _load_fusion(rp, tp, y), num_parallel_calls=AUTOTUNE)

    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)

    ds = ds.batch(BATCH).prefetch(AUTOTUNE)
    return ds

# ---------------------------
# 6) Late-fusion model
# ---------------------------
def build_late_fusion_model(lr=LR):
    # RGB branch (MobileNetV2 feature extractor)
    rgb_in = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3), name="rgb")
    x_rgb = tf.keras.applications.mobilenet_v2.preprocess_input(rgb_in * 255.0)
    base_rgb = tf.keras.applications.MobileNetV2(
        include_top=False, weights="imagenet", input_tensor=x_rgb
    )
    base_rgb.trainable = False
    x_rgb = base_rgb.output
    x_rgb = tf.keras.layers.GlobalAveragePooling2D()(x_rgb)

    # Thermal branch (small CNN)
    th_in = tf.keras.Input((IMG_SIZE, IMG_SIZE, 1), name="th")
    x_th = tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu")(th_in)
    x_th = tf.keras.layers.MaxPool2D()(x_th)
    x_th = tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu")(x_th)
    x_th = tf.keras.layers.MaxPool2D()(x_th)
    x_th = tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu")(x_th)
    x_th = tf.keras.layers.GlobalAveragePooling2D()(x_th)

    # Fusion head
    x = tf.keras.layers.Concatenate()([x_rgb, x_th])
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.35)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs=[rgb_in, th_in], outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
        ]
    )
    return model

# ---------------------------
# 7) Metrics helper + threshold search
# ---------------------------
def best_thr_for_accuracy(y_true, y_prob):
    ths = np.linspace(0.05, 0.95, 181)
    best_t, best_acc = 0.5, -1
    for t in ths:
        acc = accuracy_score(y_true, (y_prob >= t).astype(int))
        if acc > best_acc:
            best_acc = acc
            best_t = float(t)
    return best_t, best_acc

def compute_metrics(y_true, y_prob, thr):
    y_pred = (y_prob >= thr).astype(int)
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "precision": float(precision_score(y_true, y_pred, zero_division=0)),
        "recall": float(recall_score(y_true, y_pred, zero_division=0)),
        "f1": float(f1_score(y_true, y_pred, zero_division=0)),
        "roc_auc": float(roc_auc_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
        "pr_auc": float(average_precision_score(y_true, y_prob)) if len(np.unique(y_true)) > 1 else float("nan"),
        "cm": confusion_matrix(y_true, y_pred, labels=[0,1])
    }

# ---------------------------
# 8) K-Fold Training + OOF
# ---------------------------
fusion_oof_rows = []
fold_metrics = []
cm_sum = np.zeros((2,2), dtype=int)

for f in range(N_SPLITS):
    tr = df_eval[df_eval["fold"] != f].copy()
    va = df_eval[df_eval["fold"] == f].copy()

    ds_tr = make_ds_fusion(tr, training=True)
    ds_va = make_ds_fusion(va, training=False)

    # class_weight
    y_tr = tr["y_final"].values
    classes = np.array([0,1])
    cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
    class_weight = {0: float(cw[0]), 1: float(cw[1])}

    model = build_late_fusion_model(lr=LR)

    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
    ]

    model.fit(
        ds_tr,
        validation_data=ds_va,
        epochs=EPOCHS,
        verbose=0,
        class_weight=class_weight,
        callbacks=callbacks,
    )

    y_prob = model.predict(ds_va, verbose=0).ravel()
    y_true = va["y_final"].values.astype(int)

    thr_fold, _ = best_thr_for_accuracy(y_true, y_prob)

    m = compute_metrics(y_true, y_prob, thr_fold)
    m_out = {k:v for k,v in m.items() if k != "cm"}
    m_out["thr_used"] = float(thr_fold)
    m_out["fold"] = int(f)
    m_out["n_val"] = int(len(va))
    fold_metrics.append(m_out)
    cm_sum += m["cm"]

    for pid, fid, yt, yp in zip(va["patch_id"].astype(str).values,
                                va["frame_id"].astype(str).values,
                                y_true, y_prob):
        fusion_oof_rows.append({
            "patch_id": pid,
            "frame_id": fid,
            "fold": int(f),
            "y_true": int(yt),
            "y_prob": float(yp),
            "thr_used": float(thr_fold),
            "y_pred": int(yp >= thr_fold),
            "label_source": "manual_or_autoconf"
        })

# ---------------------------
# 9) Report + Save
# ---------------------------
met_df = pd.DataFrame(fold_metrics).sort_values("fold").reset_index(drop=True)
print("\nFUSION (manual+auto_confident) per-fold:")
display(met_df)

print("\nMean ± Std:")
display(met_df.drop(columns=["fold"]).agg(["mean","std"]))

print("\nConfusion Matrix (aggregated):\n", cm_sum)

oof_df = pd.DataFrame(fusion_oof_rows)
oof_df.to_csv(OUT_OOF, index=False)
met_df.to_csv(OUT_MET, index=False)

print("\n✅ Saved OOF:", OUT_OOF)
print("✅ Saved metrics:", OUT_MET)
print("OOF shape:", oof_df.shape)


# TAHAP 6 - Final Training & Packaging untuk Deployment

## 6.1 Siapkan Dataset Final untuk Deployment

In [ ]:
# ============================================
# TAHAP 6.1 — FINAL DEPLOY SET (manual + auto_confident)
# ============================================

DEPLOY_SOURCES = ["manual", "auto_confident"]

df_final = df_train[df_train["label_source"].isin(DEPLOY_SOURCES)].copy()

print("FINAL DEPLOY rows:", len(df_final))
print("Frames:", df_final["frame_id"].nunique())
print("Counts per label_source:")
display(df_final["label_source"].value_counts())
print("Label dist:")
display(df_final["y_final"].value_counts())

# dataset final (training only)
ds_final = make_ds(df_final, training=True)
print("✅ ds_final ready")


## 6.2 Final Training (Train sekali, pakai semua data deploy)

In [ ]:
# ============================================
# TAHAP 6 — FINAL TRAIN & SAVE (RGB-only)
# Dataset: patches_all_final_labeled_topk_10pct.csv
# Subset : manual + auto_confident
# Split  : GroupShuffleSplit by frame_id (anti leakage)
# Output : 1 file .keras untuk deployment
# ============================================

import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils.class_weight import compute_class_weight

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

FIX_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")
FINAL_LABELED_CSV = FIX_ROOT / "labels" / "final" / "patches_all_final_labeled_topk_10pct.csv"

MODEL_DIR = FIX_ROOT / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

OUT_RGB = MODEL_DIR / "model_rgb_final_topk10_manual_autoconf.keras"
TMP_BEST = MODEL_DIR / "tmp_best_rgb.keras"

IMG_SIZE = 128
BATCH = 64
LR_HEAD = 1e-4
LR_FT   = 5e-5
EPOCHS_HEAD = 30
EPOCHS_FT   = 10
PATIENCE_HEAD = 4
PATIENCE_FT   = 3
AUTOTUNE = tf.data.AUTOTUNE

# -------- load df --------
df = pd.read_csv(FINAL_LABELED_CSV)

for col in ["keep_for_model", "qc_canopy_ok"]:
    if col in df.columns:
        df = df[df[col] == True].copy()

df = df[df["label_source"].isin(["manual","auto_confident"])].copy()
df["y_final"] = pd.to_numeric(df["y_final"], errors="coerce")
df = df[df["y_final"].isin([0,1])].copy()
df["y_final"] = df["y_final"].astype(int)

# -------- split by frame_id (80/20) --------
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=SEED)
tr_idx, va_idx = next(gss.split(df, df["y_final"], groups=df["frame_id"]))
tr = df.iloc[tr_idx].copy().reset_index(drop=True)
va = df.iloc[va_idx].copy().reset_index(drop=True)

print("Train:", len(tr), "Val:", len(va))
print("Train dist:", tr["y_final"].value_counts().to_dict())
print("Val dist:", va["y_final"].value_counts().to_dict())

# -------- class_weight --------
y_tr = tr["y_final"].values
cw = compute_class_weight(class_weight="balanced", classes=np.array([0,1]), y=y_tr)
class_weight = {0: float(cw[0]), 1: float(cw[1])}
print("class_weight:", class_weight)

# -------- tf.data RGB --------
def tf_load_rgb(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_ds_rgb(df_part, training):
    paths = df_part["rgb_patch_path"].astype(str).values
    labels = df_part["y_final"].astype(int).values
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def _map(p, y):
        img = tf_load_rgb(p)
        return img, tf.cast(y, tf.float32)

    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(BATCH).prefetch(AUTOTUNE)

# -------- model RGB-only --------
def build_rgb_only_model(lr, train_base=False):
    base = tf.keras.applications.MobileNetV2(
        include_top=False, weights="imagenet",
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base.trainable = bool(train_base)

    inp = tf.keras.Input((IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inp * 255.0)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="roc_auc"),
                 tf.keras.metrics.AUC(curve="PR", name="pr_auc")]
    )
    return model, base

ds_tr = make_ds_rgb(tr, training=True)
ds_va = make_ds_rgb(va, training=False)

# Stage A: head
model, base = build_rgb_only_model(LR_HEAD, train_base=False)
cb_head = [
    tf.keras.callbacks.ModelCheckpoint(str(TMP_BEST), monitor="val_loss", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE_HEAD, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
]
model.fit(ds_tr, validation_data=ds_va, epochs=EPOCHS_HEAD, verbose=1,
          class_weight=class_weight, callbacks=cb_head)

# Stage B: finetune last 30 layers
base.trainable = True
for lyr in base.layers[:-30]:
    lyr.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(LR_FT),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="roc_auc"),
             tf.keras.metrics.AUC(curve="PR", name="pr_auc")]
)
cb_ft = [
    tf.keras.callbacks.ModelCheckpoint(str(TMP_BEST), monitor="val_loss", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE_FT, restore_best_weights=True),
]
model.fit(ds_tr, validation_data=ds_va, epochs=EPOCHS_FT, verbose=1,
          class_weight=class_weight, callbacks=cb_ft)

best = tf.keras.models.load_model(str(TMP_BEST))
best.save(str(OUT_RGB))
print("✅ Saved RGB final:", OUT_RGB)


In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
# ============================================
# TAHAP 6 — FINAL TRAIN & SAVE (FUSION)
# Dataset: patches_all_final_labeled_topk_10pct.csv
# Subset : manual + auto_confident
# Split  : GroupShuffleSplit by frame_id
# Output : 1 file .keras untuk deployment (fusion)
# ============================================

import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils.class_weight import compute_class_weight

# ---------------- config ----------------
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

FIX_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")
FINAL_LABELED_CSV = FIX_ROOT / "labels" / "final" / "patches_all_final_labeled_topk_10pct.csv"

MODEL_DIR = FIX_ROOT / "models"
MODEL_DIR.mkdir(parents=True, exist_ok=True)

OUT_FUSION = MODEL_DIR / "model_fusion_final_topk10_manual_autoconf.keras"
TMP_BEST   = MODEL_DIR / "tmp_best_fusion.keras"

IMG_SIZE = 128
BATCH = 64
LR_HEAD = 1e-4
LR_FT   = 5e-5
EPOCHS_HEAD = 30
EPOCHS_FT   = 10
PATIENCE_HEAD = 4
PATIENCE_FT   = 3
AUTOTUNE = tf.data.AUTOTUNE

# ---------------- load & filter df ----------------
df = pd.read_csv(FINAL_LABELED_CSV)

for col in ["keep_for_model", "qc_canopy_ok"]:
    if col in df.columns:
        df = df[df[col] == True].copy()

df = df[df["label_source"].isin(["manual", "auto_confident"])].copy()
df["y_final"] = pd.to_numeric(df["y_final"], errors="coerce")
df = df[df["y_final"].isin([0, 1])].copy()
df["y_final"] = df["y_final"].astype(int)

# ---------------- split by frame_id ----------------
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=SEED)
tr_idx, va_idx = next(gss.split(df, df["y_final"], groups=df["frame_id"]))
tr = df.iloc[tr_idx].reset_index(drop=True)
va = df.iloc[va_idx].reset_index(drop=True)

print("Train:", len(tr), "Val:", len(va))
print("Train dist:", tr["y_final"].value_counts().to_dict())
print("Val dist:", va["y_final"].value_counts().to_dict())

# ---------------- class weight ----------------
cw = compute_class_weight(
    class_weight="balanced",
    classes=np.array([0, 1]),
    y=tr["y_final"].values
)
class_weight = {0: float(cw[0]), 1: float(cw[1])}
print("class_weight:", class_weight)

# ---------------- tf.data loaders ----------------
def tf_load_rgb(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img

def tf_load_thermal_npy(path):
    th = tf.numpy_function(
        lambda p: np.load(p.decode("utf-8")).astype(np.float32),
        [path],
        tf.float32
    )
    th.set_shape([None, None])
    th = tf.expand_dims(th, -1)
    th = tf.image.resize(th, (IMG_SIZE, IMG_SIZE))
    th = (th - tf.reduce_min(th)) / (tf.reduce_max(th) - tf.reduce_min(th) + 1e-6)
    return th

def make_ds_fusion(df_part, training):
    rgb_paths = df_part["rgb_patch_path"].astype(str).values
    th_paths  = df_part["thermal_patch_path"].astype(str).values
    labels    = df_part["y_final"].values

    ds = tf.data.Dataset.from_tensor_slices((rgb_paths, th_paths, labels))

    def _map(rp, tp, y):
        rgb = tf_load_rgb(rp)          # (H,W,3)
        th  = tf_load_thermal_npy(tp)  # (H,W,1)
        x   = tf.concat([rgb, th], axis=-1)  # (H,W,4)
        return x, tf.cast(y, tf.float32)

    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(BATCH).prefetch(AUTOTUNE)

ds_tr = make_ds_fusion(tr, training=True)
ds_va = make_ds_fusion(va, training=False)

# ---------------- fusion model ----------------
def build_fusion_model(lr, train_base=False):
    inp = tf.keras.Input((IMG_SIZE, IMG_SIZE, 4))

    # adaptor 4 → 3 (biar bisa pakai ImageNet)
    x = tf.keras.layers.Conv2D(3, 1, padding="same", name="adapt_4to3")(inp)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x * 255.0)

    base = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base.trainable = bool(train_base)

    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.25)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    model = tf.keras.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc")
        ]
    )
    return model, base

# ---------------- Stage A: head ----------------
model, base = build_fusion_model(LR_HEAD, train_base=False)

cb_head = [
    tf.keras.callbacks.ModelCheckpoint(str(TMP_BEST), monitor="val_loss", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE_HEAD, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6),
]

model.fit(
    ds_tr,
    validation_data=ds_va,
    epochs=EPOCHS_HEAD,
    verbose=1,
    class_weight=class_weight,
    callbacks=cb_head
)

# ---------------- Stage B: finetune ----------------
base.trainable = True
for lyr in base.layers[:-30]:
    lyr.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(LR_FT),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(name="roc_auc"),
        tf.keras.metrics.AUC(curve="PR", name="pr_auc")
    ]
)

cb_ft = [
    tf.keras.callbacks.ModelCheckpoint(str(TMP_BEST), monitor="val_loss", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE_FT, restore_best_weights=True),
]

model.fit(
    ds_tr,
    validation_data=ds_va,
    epochs=EPOCHS_FT,
    verbose=1,
    class_weight=class_weight,
    callbacks=cb_ft
)

# ---------------- save final ----------------
best = tf.keras.models.load_model(str(TMP_BEST))
best.save(str(OUT_FUSION))
print("✅ Saved FUSION final:", OUT_FUSION)


In [ ]:
# ============================================
# TAHAP 6.2 — FINAL TRAINING (Late Fusion) for DEPLOY
# ============================================

import tensorflow as tf
import numpy as np
from pathlib import Path

OUT_FINAL = MODEL_DIR / "model_final_deploy_flask_fix.keras"

# model baru
model_final = build_late_fusion_model(img_size=IMG_SIZE, lr=1e-4)

# callback: simpan model terbaik berdasar training loss (karena tidak ada val)
cb = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(OUT_FINAL),
        monitor="loss", mode="min",
        save_best_only=True, verbose=1
    )
]

# training
EPOCHS = 20
hist = model_final.fit(ds_final, epochs=EPOCHS, callbacks=cb, verbose=1)

print("✅ Saved final deploy model:", OUT_FINAL)


## 6.3 Fine-tuning ringan MobileNet

In [ ]:
# ============================================
# TAHAP 6.3 — OPTIONAL: Fine-tune last layers (small LR)
# ============================================

# load kembali best model dari 6.2 (biar starting point yang bagus)
model_ft = tf.keras.models.load_model(str(OUT_FINAL))

# buka sebagian layer backbone mobilenet (rgb branch)
# cari layer mobilenetv2 (nama bisa beda, tapi di summary biasanya "mobilenetv2_1.00_128")
for layer in model_ft.layers:
    if "mobilenetv2" in layer.name.lower():
        backbone = layer
        break
else:
    backbone = None

print("Backbone found:", backbone.name if backbone else "NOT FOUND")

if backbone is not None:
    backbone.trainable = True
    # freeze sebagian besar, unfreeze ~20 layer terakhir
    for l in backbone.layers[:-20]:
        l.trainable = False

    model_ft.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
            tf.keras.metrics.Recall(name="recall"),
            tf.keras.metrics.Precision(name="precision"),
        ]
    )

    OUT_FINAL_FT = MODEL_DIR / "model_final_deploy_ft.keras"
    cb_ft = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=str(OUT_FINAL_FT),
            monitor="loss", mode="min",
            save_best_only=True, verbose=1
        )
    ]
    model_ft.fit(ds_final, epochs=10, callbacks=cb_ft, verbose=1)
    print("✅ Saved fine-tuned deploy model:", OUT_FINAL_FT)
else:
    print("Skip fine-tune (backbone not detected).")


## 6.4 Simpan “Inference Config”

In [ ]:
# ============================================
# TAHAP 6.4 — SAVE INFERENCE CONFIG (json)
# ============================================

import json
from datetime import datetime

INFER_CFG = {
    "created_at": datetime.now().isoformat(),
    "img_size": int(IMG_SIZE),
    "batch": int(BATCH),
    "threshold": 0.5,              # default classification threshold
    "label_sources_deploy": DEPLOY_SOURCES,
    "thermal_norm": {
        "method": "percentile_minmax",
        "p1": 1,
        "p99": 99,
        "within": "canopy_mask_v2"
    },
    "canopy_mask": {
        "name": "canopy_mask_v2_from_rgb",
        "exg_p2_p98": True,
        "green_hsv": {"H": [25,95], "S_min": 30, "V_min": 30},
        "green_exg_thr": 0.35,
        "morph": {"open": 1, "close": 2, "kernel": [5,5]}
    },
    "note": "Final deploy training uses all deploy-set (manual + auto_confident). Evaluation reported separately using GOLD K-fold."
}

CFG_PATH = MODEL_DIR / "inference_configv2.json"
with open(CFG_PATH, "w") as f:
    json.dump(INFER_CFG, f, indent=2)

print("✅ Saved config:", CFG_PATH)


## 6.5 Sanity Check Inference

In [ ]:
# ============================================
# TAHAP 6.5 — SANITY INFERENCE CHECK
# ============================================

import tensorflow as tf
import numpy as np

# pilih model mana yang dipakai
MODEL_PATH = str(OUT_FINAL)  # atau str(OUT_FINAL_FT) kalau fine-tuning dipakai
m = tf.keras.models.load_model(MODEL_PATH)
print("Loaded:", MODEL_PATH)

# ambil sampel kecil
df_sample = df_final.sample(16, random_state=SEED).copy()
ds_sample = make_ds(df_sample, training=False)

y_prob = m.predict(ds_sample).ravel()
y_true = df_sample["y_final"].astype(int).values
y_pred = (y_prob >= 0.5).astype(int)

print("Sample preds (first 10):")
for i in range(min(10, len(y_prob))):
    print(df_sample.iloc[i]["patch_id"], "true=", y_true[i], "prob=", float(y_prob[i]), "pred=", int(y_pred[i]))

print("✅ Sanity check done.")


In [ ]:
# ============================================
# TAHAP 6.5 — SANITY INFERENCE CHECK
# ============================================

import tensorflow as tf
import numpy as np

# pilih model mana yang dipakai
MODEL_PATH = str(OUT_FINAL_FT)  # atau str(OUT_FINAL_FT) kalau fine-tuning dipakai
m = tf.keras.models.load_model(MODEL_PATH)
print("Loaded:", MODEL_PATH)

# ambil sampel kecil
df_sample = df_final.sample(16, random_state=SEED).copy()
ds_sample = make_ds(df_sample, training=False)

y_prob = m.predict(ds_sample).ravel()
y_true = df_sample["y_final"].astype(int).values
y_pred = (y_prob >= 0.5).astype(int)

print("Sample preds (first 10):")
for i in range(min(10, len(y_prob))):
    print(df_sample.iloc[i]["patch_id"], "true=", y_true[i], "prob=", float(y_prob[i]), "pred=", int(y_pred[i]))

print("✅ Sanity check done.")


In [ ]:
# =====================================================
# FINAL TRAINING — RGB ONLY (DEPLOY-set) + SAVE .keras
# =====================================================

import tensorflow as tf
from pathlib import Path
import pandas as pd

# --- 0) Output path
OUT_RGB_DEPLOY = MODEL_DIR / "model_final_deploy_rgb_onlyv2.keras"
print("Will save to:", OUT_RGB_DEPLOY)

# --- 1) Cari dataframe deploy (pilih yang paling masuk akal)
# Prioritas: deploy_df / df_deploy / train_df / df_train
candidates = [
    "deploy_df", "df_deploy",
    "train_df", "df_train",
    "df_train_full", "train_df_full",
    "df_all", "df_final", "final_df"
]

df_deploy = None
for name in candidates:
    if name in globals() and isinstance(globals()[name], pd.DataFrame):
        tmp = globals()[name]
        # syarat minimal harus punya label y_final
        if "y_final" in tmp.columns:
            df_deploy = tmp.copy()
            print(f"✅ Using dataframe: {name} | rows={len(df_deploy)}")
            break

# Kalau belum ketemu, kamu SET manual di sini:
# df_deploy = df_train.copy()

assert df_deploy is not None, "❌ Tidak menemukan dataframe deploy. Set manual: df_deploy = <df kamu>"

# --- 2) (Opsional tapi disarankan) filter hanya label yang kamu anggap 'deploy'
# Kalau dataset deploy kamu memang gabungan manual + auto_confident:
# - keep semuanya, atau
# - keep auto_confident + manual saja (buang yang noisy/uncertain)
if "label_source" in df_deploy.columns:
    # default: keep manual + auto_confident (yang kamu pakai untuk training deploy biasanya)
    keep = df_deploy["label_source"].isin(["manual", "auto_confident"])
    if keep.any():
        df_deploy = df_deploy[keep].copy()
        print("Filtered label_source to {manual, auto_confident} -> rows:", len(df_deploy))

print("DEPLOY label dist:")
print(df_deploy["y_final"].value_counts(dropna=False))

# --- 3) Dataset RGB-only: reuse make_ds(df, training=...) yang sudah kamu punya
# make_ds harus menghasilkan ((rgb, th), y) atau dict {'rgb','th'}
def make_ds_rgb_only(df, training):
    ds = make_ds(df, training=training)

    def take_rgb(x, y):
        # case A: ((rgb, th), y)
        if isinstance(x, (tuple, list)):
            return x[0], y
        # case B: {'rgb':..., 'th':...}
        if isinstance(x, dict):
            return x["rgb"], y
        # case C: already rgb
        return x, y

    return ds.map(take_rgb, num_parallel_calls=tf.data.AUTOTUNE)

ds_train = make_ds_rgb_only(df_deploy, training=True)

# --- 4) Build RGB-only model (MobileNetV2 baseline)
def build_rgb_only_model(img_size=128, lr=1e-4):
    inp = tf.keras.Input((img_size, img_size, 3), name="rgb")
    base = tf.keras.applications.MobileNetV2(
        include_top=False, input_tensor=inp, weights="imagenet"
    )
    x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
    x = tf.keras.layers.Dropout(0.2)(x)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

    m = tf.keras.Model(inp, out, name="rgb_only_deploy")
    m.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.BinaryAccuracy(name="accuracy", threshold=0.5),
            tf.keras.metrics.AUC(curve="ROC", name="roc_auc"),
            tf.keras.metrics.AUC(curve="PR", name="pr_auc"),
            tf.keras.metrics.Precision(name="precision", thresholds=0.5),
            tf.keras.metrics.Recall(name="recall", thresholds=0.5),
        ],
    )
    return m

model_rgb_deploy = build_rgb_only_model(img_size=IMG_SIZE, lr=1e-4)
model_rgb_deploy.summary()

# --- 5) Train FINAL (deploy) — tidak pakai K-Fold
# (kalau kamu punya ds_val deploy, boleh tambah validation_data=..., tapi ini opsional untuk deploy)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3, restore_best_weights=True)
]

history = model_rgb_deploy.fit(
    ds_train,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

# --- 6) SAVE
model_rgb_deploy.save(OUT_RGB_DEPLOY)
print("✅ Saved RGB-only DEPLOY model:", OUT_RGB_DEPLOY)


# TAHAP 7 - Flask App & DSS Implementation

## Imports + Paths + Load Model & Config

In [ ]:
# ============================================
# TAHAP 7.1 — Core Inference Setup (Colab)
# - Load model_final_deploy.keras
# - Load inference_config.json
# ============================================

import os, json, math
import numpy as np
import pandas as pd
from pathlib import Path
import cv2
import tensorflow as tf

# ---- Paths (sesuaikan kalau perlu) ----
MODEL_DIR = FIX_ROOT / "models"
MODEL_PATH = MODEL_DIR / "model_final_deploy.keras"   # gunakan yg kamu pilih
CFG_PATH   = MODEL_DIR / "inference_config.json"

assert MODEL_PATH.exists(), f"Model not found: {MODEL_PATH}"
assert CFG_PATH.exists(),   f"Config not found: {CFG_PATH}"

# ---- Load config ----
with open(CFG_PATH, "r") as f:
    INFER_CFG = json.load(f)

THR = float(INFER_CFG.get("threshold", 0.5))
IMG_SIZE = int(INFER_CFG.get("img_size", 128))

print("✅ Loaded config:", CFG_PATH)
print("IMG_SIZE:", IMG_SIZE, "| THR:", THR)

# ---- Load model ----
model = tf.keras.models.load_model(str(MODEL_PATH))
print("✅ Loaded model:", MODEL_PATH)
print("Model inputs:", [t.shape for t in model.inputs])


## Utility: Load RGB & Thermal Frame (npy) + Align

In [ ]:
# ============================================
# TAHAP 7.2 (REPLACE) — Load + Align RGB->Thermal grid (crop+zoom)
# ============================================

import cv2, numpy as np

def center_crop(img, crop_h, crop_w):
    H, W = img.shape[:2]
    y0 = (H - crop_h) // 2
    x0 = (W - crop_w) // 2
    return img[y0:y0+crop_h, x0:x0+crop_w], (x0, y0, crop_w, crop_h)

def rgb_to_thermal_grid(rgb_bgr, th_shape_hw, zoom=2.2):
    """
    RGB wide -> crop center (zoom-in) -> resize to thermal grid (Hth,Wth)
    Return: rgb_aligned_bgr (Hth,Wth,3), roi on original RGB
    """
    Hth, Wth = th_shape_hw
    Hr, Wr = rgb_bgr.shape[:2]
    crop_h = int(Hr / zoom)
    crop_w = int(Wr / zoom)
    rgb_crop, roi = center_crop(rgb_bgr, crop_h, crop_w)
    rgb_aligned = cv2.resize(rgb_crop, (Wth, Hth), interpolation=cv2.INTER_LINEAR)
    return rgb_aligned, roi

def load_rgb_bgr(rgb_path: str) -> np.ndarray:
    rgb = cv2.imread(str(rgb_path), cv2.IMREAD_COLOR)
    if rgb is None:
        raise FileNotFoundError(rgb_path)
    return rgb  # BGR uint8

def load_thermal_npy(th_path: str) -> np.ndarray:
    th = np.load(str(th_path)).astype(np.float32)
    if th.ndim != 2:
        raise ValueError(f"Thermal must be 2D, got: {th.shape}")
    return th

print("✅ 7.2 (replace) ready")


## Patch Grid Extractor (RGB + Thermal) + Masking + Normalisasi Thermal

In [ ]:
# ============================================
# TAHAP 7.3 (REPLACE) — Extract patches on thermal grid
# ============================================

def extract_paired_patches_on_thermal_grid(rgb_aligned_bgr, th_2d, patch=128, stride=128):
    """
    rgb_aligned_bgr: (Hth,Wth,3) BGR
    th_2d          : (Hth,Wth) float32
    Return:
      rgb_batch: (N,patch,patch,3) float32 RGB 0..1
      th_batch : (N,patch,patch,1) float32 0..1 (normalized)
      meta     : list dict
    """
    H, W = th_2d.shape[:2]
    assert rgb_aligned_bgr.shape[:2] == (H, W), "RGB aligned must match thermal shape"

    rgb_list, th_list, meta = [], [], []
    for y in range(0, H - patch + 1, stride):
        for x in range(0, W - patch + 1, stride):
            rgb_p_bgr = rgb_aligned_bgr[y:y+patch, x:x+patch]
            th_p      = th_2d[y:y+patch, x:x+patch].astype(np.float32)

            # BGR->RGB then normalize
            rgb_p = cv2.cvtColor(rgb_p_bgr, cv2.COLOR_BGR2RGB)
            rgb_f = rgb_p.astype(np.float32) / 255.0

            # canopy mask from aligned RGB patch (expects uint8 RGB)
            mask = canopy_mask_v2_from_rgb(rgb_p)  # bool (patch,patch)

            # thermal canopy-only stats
            th_can = th_p.copy()
            th_can[~mask] = np.nan
            vals = th_can[np.isfinite(th_can)]
            if vals.size < 10:
                vals = th_p[np.isfinite(th_p)]

            lo = np.percentile(vals, 1) if vals.size else 0.0
            hi = np.percentile(vals, 99) if vals.size else 1.0

            th_norm = (th_p - lo) / (hi - lo + 1e-6)
            th_norm = np.clip(th_norm, 0.0, 1.0).astype(np.float32)[..., None]

            rgb_list.append(rgb_f.astype(np.float32))
            th_list.append(th_norm)
            meta.append({"x": x, "y": y, "row": y//stride, "col": x//stride})

    rgb_batch = np.stack(rgb_list, axis=0) if rgb_list else np.zeros((0,patch,patch,3), np.float32)
    th_batch  = np.stack(th_list, axis=0)  if th_list  else np.zeros((0,patch,patch,1), np.float32)
    return rgb_batch, th_batch, meta

print("✅ 7.3 (replace) ready")


## Predict patches (Fusion model)

In [ ]:
# ============================================
# TAHAP 7.5 (REPLACE) — Build heatmap on thermal grid
# ============================================

def build_heatmap_grid(meta, y_prob, H, W, patch=128, stride=128):
    n_rows = 1 + (H - patch)//stride
    n_cols = 1 + (W - patch)//stride
    grid = np.zeros((n_rows, n_cols), dtype=np.float32)
    for m, p in zip(meta, y_prob):
        grid[m["row"], m["col"]] = p
    heat = cv2.resize(grid, (W, H), interpolation=cv2.INTER_NEAREST)  # (H,W)
    return grid, heat

print("✅ 7.5 (replace) ready")


## Heatmap + DSS decision

In [ ]:
# ============================================
# TAHAP 7.5 — Heatmap + DSS decision
# ============================================

def build_heatmap(meta, y_prob, H, W, patch=128, stride=128):
    # hitung grid size
    n_rows = 1 + (H - patch)//stride
    n_cols = 1 + (W - patch)//stride

    grid = np.zeros((n_rows, n_cols), dtype=np.float32)
    for m, p in zip(meta, y_prob):
        grid[m["row"], m["col"]] = p

    # upscale grid ke ukuran frame (biar bisa di-overlay)
    heat = cv2.resize(grid, (W, H), interpolation=cv2.INTER_NEAREST)
    return grid, heat

def dss_decision(y_prob, thr=0.5):
    if y_prob.size == 0:
        return {"stress_ratio": 0.0, "dss_status": "NO_DATA", "recommended_action": "No data"}

    stress_ratio = float((y_prob >= thr).mean())

    if stress_ratio < 0.15:
        return {"stress_ratio": stress_ratio, "dss_status": "LOW_STRESS", "recommended_action": "No action needed"}
    elif stress_ratio < 0.35:
        return {"stress_ratio": stress_ratio, "dss_status": "MED_STRESS", "recommended_action": "Field inspection / localized check"}
    else:
        return {"stress_ratio": stress_ratio, "dss_status": "HIGH_STRESS", "recommended_action": "Immediate action: targeted irrigation/fertilization check"}

print("✅ 7.5 heatmap + DSS ready")


In [ ]:
## MAIN: predict_frame (Fusion)

In [ ]:
# ============================================
# TAHAP 7.6 (REPLACE) — MAIN predict_frame on thermal grid
# ============================================

def predict_frame_thermal_grid(rgb_path: str, thermal_npy_path: str,
                              zoom=2.2, patch=128, stride=128, thr=None,
                              save_dir=None):
    thr = THR if thr is None else float(thr)

    rgb_bgr = load_rgb_bgr(rgb_path)
    th = load_thermal_npy(thermal_npy_path)

    # Align RGB -> thermal grid
    rgb_aligned_bgr, roi = rgb_to_thermal_grid(rgb_bgr, th.shape[:2], zoom=zoom)

    Hth, Wth = th.shape[:2]
    rgb_batch, th_batch, meta = extract_paired_patches_on_thermal_grid(
        rgb_aligned_bgr, th, patch=patch, stride=stride
    )

    y_prob = model.predict([rgb_batch, th_batch], verbose=0).ravel().astype(np.float32)

    # DSS decision (same as before)
    dss = dss_decision(y_prob, thr=thr)

    grid, heat = build_heatmap_grid(meta, y_prob, Hth, Wth, patch=patch, stride=stride)

    out = {
        "mode": "FUSION_THERMAL_GRID",
        "zoom": float(zoom),
        "roi_on_rgb": roi,     # (x0,y0,w,h) in original RGB
        "thermal_shape": (int(Hth), int(Wth)),
        "n_patches": int(len(meta)),
        "threshold": float(thr),
        **dss,
        "patch_results": [{**m, "p_stress": float(p), "pred": int(p >= thr)} for m, p in zip(meta, y_prob)],
        "heatmap_path": None
    }

    # Save outputs
    if save_dir is not None:
        save_dir = Path(save_dir); save_dir.mkdir(parents=True, exist_ok=True)

        # heatmap grayscale
        heat_u8 = np.clip(heat * 255.0, 0, 255).astype(np.uint8)
        hm_path = save_dir / (Path(rgb_path).stem + f"_heatmap_zoom{zoom}.png")
        cv2.imwrite(str(hm_path), heat_u8)
        out["heatmap_path"] = str(hm_path)

        # overlay preview on aligned RGB
        rgb_show = cv2.cvtColor(rgb_aligned_bgr, cv2.COLOR_BGR2RGB)
        overlay = rgb_show.copy()
        # simple alpha blend using heat as mask
        heat3 = np.stack([heat_u8]*3, axis=-1)
        overlay = cv2.addWeighted(overlay, 1.0, heat3, 0.45, 0.0)
        ov_path = save_dir / (Path(rgb_path).stem + f"_overlay_zoom{zoom}.png")
        cv2.imwrite(str(ov_path), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
        out["overlay_path"] = str(ov_path)

    return out

print("✅ 7.6 (replace) ready")


In [ ]:
# ============================================
# TAHAP 7.7 — TEST RUN
# ============================================

RGB_PATH = "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/data/raw/rgb/DJI_20251223095727_0030_V.JPG"
TH_PATH  = "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/data/raw/thermal/DJI_20251223095727_0030_T.npy"

out2 = predict_frame_thermal_grid(
    rgb_path=RGB_PATH,
    thermal_npy_path=TH_PATH,
    zoom=2.2,
    patch=128,
    stride=128,
    thr=THR,
    save_dir=str(FIX_ROOT / "dss_outputs")
)

print("Mode:", out2["mode"])
print("n_patches:", out2["n_patches"])
print("stress_ratio:", out2["stress_ratio"])
print("dss_status:", out2["dss_status"])
print("overlay_path:", out2.get("overlay_path"))


In [ ]:
# ============================================
# TAHAP 7.10 — DSS scoring (extent + severity)
# ============================================

import numpy as np

probs = np.array([r["p_stress"] for r in out2["patch_results"]], dtype=np.float32)

thr = out2["threshold"]
pred = (probs >= thr)

extent = float(pred.mean())                                # proporsi patch stress
severity_mean = float(probs[pred].mean()) if pred.any() else 0.0
severity_p90  = float(np.percentile(probs, 90))

print("extent:", extent)
print("severity_mean(stress patches):", severity_mean)
print("severity_p90(all patches):", severity_p90)

# ---- DSS rule (lebih stabil dari cuma ratio) ----
# Kamu bisa tweak angka ini nanti berdasarkan evaluasi / expert judgement
if extent < 0.25 and severity_p90 < 0.75:
    status = "LOW_STRESS"
    action = "No action needed; continue monitoring."
elif extent < 0.50 and severity_p90 < 0.85:
    status = "MODERATE_STRESS"
    action = "Check irrigation & nutrients in highlighted zones."
else:
    status = "HIGH_STRESS"
    action = "Immediate action: targeted field inspection + irrigation/fertilization adjustment."

print("\nDSS_STATUS_V2:", status)
print("recommended_action:", action)


In [ ]:
# ============================================
# TAHAP 7.9 (REVISED) — Show saved overlay (thermal-grid aligned)
# ============================================

import matplotlib.pyplot as plt
import cv2

ov_path = out2.get("overlay_path", None)
hm_path = out2.get("heatmap_path", None)

assert ov_path is not None, "overlay_path tidak ada. Pastikan save_dir dipakai saat predict_frame_thermal_grid."

overlay_bgr = cv2.imread(ov_path, cv2.IMREAD_COLOR)
assert overlay_bgr is not None, f"Gagal baca overlay: {ov_path}"
overlay_rgb = cv2.cvtColor(overlay_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10,7))
plt.imshow(overlay_rgb)
plt.title(f"Overlay (thermal-grid aligned) | zoom={out2.get('zoom')} | n_patches={out2.get('n_patches')}")
plt.axis("off")
plt.show()

# (opsional) tampilkan heatmap grayscale juga
if hm_path is not None:
    heat_u8 = cv2.imread(hm_path, cv2.IMREAD_GRAYSCALE)
    if heat_u8 is not None:
        plt.figure(figsize=(6,4))
        plt.imshow(heat_u8, cmap="gray")
        plt.title("Heatmap grayscale (saved)")
        plt.axis("off")


In [ ]:
# ============================================
# TAHAP 7.10 — DSS scoring (extent + severity)
# ============================================

import numpy as np

probs = np.array([r["p_stress"] for r in out2["patch_results"]], dtype=np.float32)

thr = out2["threshold"]
pred = (probs >= thr)

extent = float(pred.mean())                                # proporsi patch stress
severity_mean = float(probs[pred].mean()) if pred.any() else 0.0
severity_p90  = float(np.percentile(probs, 90))

print("extent:", extent)
print("severity_mean(stress patches):", severity_mean)
print("severity_p90(all patches):", severity_p90)

# ---- DSS rule (lebih stabil dari cuma ratio) ----
# Kamu bisa tweak angka ini nanti berdasarkan evaluasi / expert judgement
if extent < 0.25 and severity_p90 < 0.75:
    status = "LOW_STRESS"
    action = "No action needed; continue monitoring."
elif extent < 0.50 and severity_p90 < 0.85:
    status = "MODERATE_STRESS"
    action = "Check irrigation & nutrients in highlighted zones."
else:
    status = "HIGH_STRESS"
    action = "Immediate action: targeted field inspection + irrigation/fertilization adjustment."

print("\nDSS_STATUS_V2:", status)
print("recommended_action:", action)


In [ ]:
# ============================================
# TAHAP 7.10b — DSS Rule V3 (extent + severity + hotspot)
# ============================================

import numpy as np

probs = np.array([r["p_stress"] for r in out2["patch_results"]], dtype=np.float32)
thr = float(out2.get("threshold", 0.5))
pred = (probs >= thr)

extent = float(pred.mean())
sev_mean = float(probs[pred].mean()) if pred.any() else 0.0
p90 = float(np.percentile(probs, 90))
p95 = float(np.percentile(probs, 95))
p99 = float(np.percentile(probs, 99))

print("extent:", extent)
print("sev_mean:", sev_mean, "| p90:", p90, "| p95:", p95, "| p99:", p99)

# ---- DSS V3 ----
# tweakable thresholds (awal yang masuk akal)
HOTSPOT = (p99 >= 0.90)                    # ada area sangat yakin stress
SEVERE_WIDE = (extent >= 0.60 and p90 >= 0.80)  # luas + yakin tinggi
MOD_WIDE = (extent >= 0.50 and p90 >= 0.65)     # luas + keyakinan moderat

if SEVERE_WIDE or HOTSPOT:
    status = "HIGH_STRESS"
    action = "Immediate action: targeted field inspection + irrigation/fertilization adjustment."
elif MOD_WIDE:
    status = "MODERATE_STRESS"
    action = "Action soon: inspect highlighted zones; adjust irrigation/nutrients if confirmed."
else:
    status = "LOW_STRESS"
    action = "No immediate action; continue monitoring."

print("\nDSS_STATUS_V3:", status)
print("recommended_action:", action)


In [ ]:
# ============================================
# TAHAP 7.11 — Save DSS output JSON (single frame)
# ============================================

import json
from pathlib import Path
import numpy as np

SAVE_DIR = FIX_ROOT / "dss_outputs"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

def build_dss_v3(out):
    probs = np.array([r["p_stress"] for r in out["patch_results"]], dtype=np.float32)
    thr = float(out.get("threshold", 0.5))
    pred = probs >= thr

    extent = float(pred.mean())
    sev_mean = float(probs[pred].mean()) if pred.any() else 0.0
    p90 = float(np.percentile(probs, 90))
    p95 = float(np.percentile(probs, 95))
    p99 = float(np.percentile(probs, 99))

    HOTSPOT = (p99 >= 0.90)
    SEVERE_WIDE = (extent >= 0.60 and p90 >= 0.80)
    MOD_WIDE = (extent >= 0.50 and p90 >= 0.65)

    if SEVERE_WIDE or HOTSPOT:
        status = "HIGH_STRESS"
        action = "Immediate action: targeted field inspection + irrigation/fertilization adjustment."
    elif MOD_WIDE:
        status = "MODERATE_STRESS"
        action = "Action soon: inspect highlighted zones; adjust irrigation/nutrients if confirmed."
    else:
        status = "LOW_STRESS"
        action = "No immediate action; continue monitoring."

    return {
        "threshold": thr,
        "extent": extent,
        "severity_mean": sev_mean,
        "p90": p90,
        "p95": p95,
        "p99": p99,
        "dss_status": status,
        "recommended_action": action,
    }

# --- build summary ---
summary = build_dss_v3(out2)

# --- choose a filename (use overlay name) ---
overlay_name = Path(out2.get("overlay_path","frame")).stem
json_path = SAVE_DIR / f"{overlay_name}_dss.json"

payload = {
    "mode": out2.get("mode"),
    "zoom": out2.get("zoom"),
    "thermal_shape": out2.get("thermal_shape"),
    "n_patches": out2.get("n_patches"),
    "overlay_path": out2.get("overlay_path"),
    "heatmap_path": out2.get("heatmap_path"),
    "summary": summary,
    "patch_results": out2.get("patch_results", [])  # row,col,x,y,p_stress,pred
}

with open(json_path, "w") as f:
    json.dump(payload, f, indent=2)

print("✅ Saved:", json_path)
print("DSS:", summary["dss_status"], "| extent:", round(summary["extent"],3), "| p90:", round(summary["p90"],3), "| p99:", round(summary["p99"],3))


#

# Backup

In [ ]:
from pathlib import Path
ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")
MODEL_PATH = ROOT / "models" / "model_final_deploy_ft.keras"
EXPORT_DIR = ROOT / "exports"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

print("MODEL exists:", MODEL_PATH.exists(), MODEL_PATH)

In [ ]:
import pandas as pd

# coba beberapa kandidat path CSV (sesuaikan bila perlu)
candidates = [
    ROOT / "patches" / "meta" / "patches_all_splits.csv",
    ROOT / "patches" / "meta" / "patches_all_enriched.csv",
    ROOT / "patches" / "meta" / "patches_all.csv",
]

df = None
for p in candidates:
    if p.exists():
        df = pd.read_csv(p)
        print("Loaded:", p, "shape:", df.shape)
        break

assert df is not None, "CSV meta patch tidak ditemukan. Cek folder patches/meta di Drive."
print(df.columns)


In [ ]:
# sesuaikan nama split jika berbeda
if "split" in df.columns:
    df_dep = df[df["split"].astype(str).str.upper().isin(["DEPLOY","DEPLOYSET","DEPLOY_SET"])].copy()
else:
    # kalau tidak ada split, pakai semua yang punya label
    df_dep = df.copy()

# ambil yang labelnya ada
label_col_candidates = ["label", "y", "target", "label_final"]
label_col = next((c for c in label_col_candidates if c in df_dep.columns), None)
assert label_col is not None, "Kolom label tidak ketemu (label/y/target/label_final)."

df_dep = df_dep[df_dep[label_col].isin([0,1])].reset_index(drop=True)
print("DEPLOY rows:", len(df_dep))


# COBA


In [ ]:
import pandas as pd
from pathlib import Path

FINAL_LABELED_CSV = Path(
    "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/"
    "DatasetTerbaruFIX/labels/final/patches_all_final_labeled_topk_10pct.csv"
)

print("File exists:", FINAL_LABELED_CSV.exists())

df = pd.read_csv(FINAL_LABELED_CSV)
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())

df.head()


In [ ]:
import pandas as pd
from pathlib import Path

# =========================
# Load CSV (self-contained)
# =========================
df = pd.read_csv(
    "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/"
    "DatasetTerbaruFIX/labels/final/patches_all_final_labeled_topk_10pct.csv"
)

# =========================
# Filter manual + auto_confident
# =========================
df_eval = df[df["label_source"].isin(["manual", "auto_confident"])].copy()

df_eval["y_final"] = pd.to_numeric(df_eval["y_final"], errors="coerce")
df_eval = df_eval[df_eval["y_final"].isin([0, 1])]
df_eval["y_final"] = df_eval["y_final"].astype(int)

df_eval = df_eval.reset_index(drop=True)

print("Eval rows:", len(df_eval))  # harus 1293

# =========================
# Build RGB–Thermal pairing
# =========================
def rgb_to_thermal(rgb_path):
    return str(rgb_path).replace("/rgb/", "/thermal/")

df_pairs = pd.DataFrame({
    "RGB_path": df_eval["rgb_patch_path"].astype(str),
    "Thermal_path": df_eval["rgb_patch_path"].apply(rgb_to_thermal),
    "Label": df_eval["y_final"]
})

# =========================
# Buat kolom NAMA FILE saja
# =========================
df_pairs["RGB"] = df_pairs["RGB_path"].apply(lambda p: Path(p).name)
df_pairs["Thermal"] = df_pairs["Thermal_path"].apply(lambda p: Path(p).name)

# =========================
# Pilih kolom untuk ditampilkan
# =========================
df_show = df_pairs[["RGB", "Thermal", "Label"]]

# =========================
# TAMPILKAN: 10 awal + 10 akhir
# =========================
df_preview = pd.concat([df_show.head(10), df_show.tail(10)])

display(df_preview)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# =========================
# PATH CSV
# =========================
CSV_PATH = (
    "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/"
    "DatasetTerbaruFIX/labels/final/patches_all_final_labeled_topk_10pct.csv"
)

# =========================
# LOAD & FILTER DATA
# =========================
df = pd.read_csv(CSV_PATH)

df_eval = df[df["label_source"].isin(["manual", "auto_confident"])].copy()
df_eval["y_final"] = pd.to_numeric(df_eval["y_final"], errors="coerce")
df_eval = df_eval[df_eval["y_final"].isin([0, 1])]
df_eval["y_final"] = df_eval["y_final"].astype(int)

print("Total eval patches:", len(df_eval))  # harus 1293

# =========================
# HITUNG JUMLAH LABEL
# =========================
label_counts = df_eval["y_final"].value_counts().sort_index()  # index=0,1
normal_count = label_counts.get(0, 0)
stress_count = label_counts.get(1, 0)

# =========================
# PLOT DENGAN WARNA BERBEDA
# =========================
labels = ['Normal', 'Stress']
counts = [normal_count, stress_count]
colors = ['green', 'red']  # Normal = hijau, Stress = merah

plt.figure(figsize=(6,4))
plt.bar(labels, counts, color=colors)
plt.ylabel("Jumlah Patch")
plt.title("Perbandingan Dataset Normal vs Stress")

# Optional: tampilkan angka di atas bar
for i, count in enumerate(counts):
    plt.text(i, count + 5, str(count), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
import random

# =========================
# Paths
# =========================
FIX_ROOT = Path("/content/drive/MyDrive/rice_stress_detection_rgb_thermal/DatasetTerbaruFIX")
GOLD_CSV = FIX_ROOT / "labels" / "gold" / "labels_gold.csv"
RGB_PATCH_DIR = FIX_ROOT / "patches" / "rgb"  # folder patch RGB asli
SAVE_DIR = FIX_ROOT / "dokumentasi" / "skip"   # folder simpan patch skip
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# =========================
# Load GOLD CSV
# =========================
if not GOLD_CSV.exists():
    raise FileNotFoundError(f"Tidak ditemukan: {GOLD_CSV}")

gold = pd.read_csv(GOLD_CSV)

# normalisasi label
gold["label_manual_num"] = pd.to_numeric(gold["label_manual"], errors="coerce")
notes = gold["notes"].fillna("").astype(str) if "notes" in gold.columns else pd.Series([""]*len(gold))
is_skip_note = notes.str.contains(r"\[SKIP\]", case=False, regex=True)
is_empty_label = gold["label_manual_num"].isna()

# filter patch skip / empty
skip_patches = gold[is_empty_label | is_skip_note]
print("Total patch skip / empty:", len(skip_patches))

# ambil 20 patch random (atau semua jika <20)
sample_skip = skip_patches.sample(min(10, len(skip_patches)), random_state=42)

# =========================
# Tampilkan & Simpan patch
# =========================
plt.figure(figsize=(15,10))
possible_exts = [".jpg", ".jpeg", ".png"]

saved_count = 0
for i, row in enumerate(sample_skip.itertuples(), 1):
    rgb_path = None
    for ext in possible_exts:
        candidate = RGB_PATCH_DIR / f"{row.patch_id}{ext}"
        if candidate.exists():
            rgb_path = candidate
            break
    if rgb_path is None:
        print(f"File tidak ditemukan untuk patch_id {row.patch_id}")
        continue

    # load image
    img = cv2.imread(str(rgb_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # tampilkan
    plt.subplot(4,5,i)
    plt.imshow(img)
    plt.axis("off")
    title = f"{row.patch_id}\nEmpty:{pd.isna(row.label_manual_num)} SkipNote:{'[SKIP]' in str(row.notes)}"
    plt.title(title, fontsize=8)

    # simpan ke folder dokumentasi/skip
    save_path = SAVE_DIR / f"{row.patch_id}.jpg"
    cv2.imwrite(str(save_path), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    saved_count += 1

plt.tight_layout()
plt.show()
print(f"✅ Total patch berhasil disimpan: {saved_count}")


In [ ]:
# =========================================================
# FULL PIPELINE: load -> align (crop+zoom) -> overlay -> patches
# =========================================================

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 1) Utils
# -----------------------------
def normalize_to_u8(x, p1=1, p99=99):
    """Normalize 2D float array to uint8 for visualization."""
    x = x.astype(np.float32)
    lo, hi = np.nanpercentile(x, p1), np.nanpercentile(x, p99)
    x = np.clip((x - lo) / (hi - lo + 1e-6), 0, 1)
    return (x * 255).astype(np.uint8)

def center_crop(img, crop_h, crop_w):
    H, W = img.shape[:2]
    y0 = (H - crop_h) // 2
    x0 = (W - crop_w) // 2
    return img[y0:y0+crop_h, x0:x0+crop_w], (x0, y0, crop_w, crop_h)

def rgb_to_thermal_grid(rgb_bgr, th_shape_hw, zoom=2.2):
    """
    RGB wide -> crop center (zoom-in) -> resize to thermal grid.
    zoom bigger => crop smaller => RGB looks more zoomed-in.
    """
    Hth, Wth = th_shape_hw
    Hr, Wr = rgb_bgr.shape[:2]
    crop_h = int(Hr / zoom)
    crop_w = int(Wr / zoom)

    rgb_crop, roi = center_crop(rgb_bgr, crop_h, crop_w)
    rgb_aligned = cv2.resize(rgb_crop, (Wth, Hth), interpolation=cv2.INTER_LINEAR)
    return rgb_aligned, roi

def show_alignment(rgb_aligned_bgr, th_2d, title_suffix=""):
    """3-panel plot: RGB aligned, Thermal, Overlay."""
    th_u8 = normalize_to_u8(th_2d)
    rgb_show = cv2.cvtColor(rgb_aligned_bgr, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12, 4))
    plt.subplot(1,3,1); plt.imshow(rgb_show); plt.title(f"RGB aligned{title_suffix}"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(th_u8, cmap="inferno"); plt.title(f"Thermal{title_suffix}"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(rgb_show); plt.imshow(th_u8, cmap="inferno", alpha=0.35); plt.title(f"Overlay{title_suffix}"); plt.axis("off")
    plt.tight_layout()
    plt.show()

def extract_paired_patches(rgb_aligned_bgr, th_2d, patch=128, stride=128):
    """
    Extract paired patches (RGB + Thermal) from same grid.
    Return:
      rgb_patches: (N, patch, patch, 3) uint8
      th_patches : (N, patch, patch) float32
      meta       : list dict per patch
    """
    H, W = th_2d.shape[:2]
    rgb_patches, th_patches, meta = [], [], []

    for y in range(0, H - patch + 1, stride):
        for x in range(0, W - patch + 1, stride):
            rgb_p = rgb_aligned_bgr[y:y+patch, x:x+patch]
            th_p  = th_2d[y:y+patch, x:x+patch]

            rgb_patches.append(rgb_p)
            th_patches.append(th_p.astype(np.float32))

            meta.append({
                "x": x, "y": y,
                "patch": patch,
                "row": y // stride,
                "col": x // stride
            })

    return np.stack(rgb_patches, axis=0), np.stack(th_patches, axis=0), meta


# -----------------------------
# 2) Paths (EDIT THIS)
# -----------------------------
rgb_path = "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/data/raw/rgb/DJI_20251223095741_0048_V.JPG"
th_path  = "/content/drive/MyDrive/rice_stress_detection_rgb_thermal/data/raw/thermal/DJI_20251223095741_0048_T.npy"

zoom = 2.2       # <-- ganti kalau perlu (2.0 / 2.2 / 2.4 / 2.6)
patch = 128
stride = 128


# -----------------------------
# 3) Load data
# -----------------------------
rgb = cv2.imread(rgb_path)   # BGR
if rgb is None:
    raise ValueError(f"RGB gagal dibaca. Cek path: {rgb_path}")

th = np.load(th_path)
if th.ndim != 2:
    raise ValueError(f"Thermal npy harus 2D. Dapat shape={th.shape}")

print("RGB shape:", rgb.shape, rgb.dtype)
print("TH  shape:", th.shape, th.dtype)


# -----------------------------
# 4) Align RGB -> thermal grid
# -----------------------------
rgb_aligned, roi = rgb_to_thermal_grid(rgb, th.shape[:2], zoom=zoom)
print("ALIGN PARAMS:", {"zoom": zoom, "roi": roi})
print("RGB aligned shape:", rgb_aligned.shape)


# -----------------------------
# 5) Sanity check overlay
# -----------------------------
show_alignment(rgb_aligned, th, title_suffix=f" (zoom={zoom})")


# -----------------------------
# 6) Extract paired patches
# -----------------------------
rgb_patches, th_patches, meta = extract_paired_patches(rgb_aligned, th, patch=patch, stride=stride)
print("RGB patches:", rgb_patches.shape, rgb_patches.dtype)
print("TH  patches:", th_patches.shape, th_patches.dtype)
print("Example meta[0]:", meta[0])


# -----------------------------
# 7) (Optional) Visualize a few random paired patches
# -----------------------------
idxs = [0, len(meta)//2, len(meta)-1] if len(meta) >= 3 else list(range(len(meta)))

plt.figure(figsize=(12, 6))
for i, idx in enumerate(idxs, 1):
    rgb_p = cv2.cvtColor(rgb_patches[idx], cv2.COLOR_BGR2RGB)
    th_u8 = normalize_to_u8(th_patches[idx])

    ax = plt.subplot(2, len(idxs), i)
    ax.imshow(rgb_p); ax.set_title(f"RGB patch #{idx}"); ax.axis("off")

    ax = plt.subplot(2, len(idxs), i + len(idxs))
    ax.imshow(th_u8, cmap="inferno"); ax.set_title(f"TH patch #{idx}"); ax.axis("off")

plt.tight_layout()
plt.show()

# =================================================
# 8) Fungsi baru: overlay dari pasangan patch RGB+TH
# =================================================
def show_paired_patch_overlay(rgb_patches, th_patches, idxs=None, alpha=0.35):
    """
    Tampilkan overlay RGB + Thermal dari patch yang sudah diekstrak.
    Args:
        rgb_patches: np.array (N, H, W, 3) uint8
        th_patches : np.array (N, H, W) float32
        idxs       : list[int] indices patch yang ingin ditampilkan, default: semua
        alpha      : float, opacity thermal overlay
    """
    N = len(rgb_patches)
    if idxs is None:
        idxs = list(range(N))

    plt.figure(figsize=(12, 4*len(idxs)))
    for i, idx in enumerate(idxs, 1):
        rgb = cv2.cvtColor(rgb_patches[idx], cv2.COLOR_BGR2RGB)
        th_u8 = normalize_to_u8(th_patches[idx])

        plt.subplot(len(idxs), 1, i)
        plt.imshow(rgb)
        plt.imshow(th_u8, cmap="inferno", alpha=alpha)
        plt.title(f"Overlay patch #{idx}")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

# -----------------------------
# Contoh penggunaan
# -----------------------------
# pilih beberapa patch acak atau tertentu
sample_idxs = [0, 10, 19] if len(rgb_patches) > 20 else list(range(len(rgb_patches)))
show_paired_patch_overlay(rgb_patches, th_patches, idxs=sample_idxs)


In [ ]:
import re
from pathlib import Path
import pandas as pd
from datetime import datetime

# =========================
# PATHS
# =========================
RAW_RGB_DIR = PROJECT_ROOT / "data" / "raw" / "rgb"
RAW_TH_DIR  = PROJECT_ROOT / "data" / "raw" / "thermal"

if not RAW_RGB_DIR.exists():
    raise FileNotFoundError(f"RAW_RGB_DIR tidak ditemukan: {RAW_RGB_DIR}")
if not RAW_TH_DIR.exists():
    raise FileNotFoundError(f"RAW_TH_DIR tidak ditemukan: {RAW_TH_DIR}")

rgb_exts = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")
rgb_paths = sorted([p for p in RAW_RGB_DIR.iterdir() if p.is_file() and p.name.endswith(rgb_exts)])
th_paths  = sorted([p for p in RAW_TH_DIR.iterdir() if p.is_file() and p.suffix.lower() == ".npy"])

print("Total RGB files   :", len(rgb_paths))
print("Total Thermal npy :", len(th_paths))

# =========================
# HELPERS
# =========================
def parse_frame_id_from_rgb_name(rgb_filename: str) -> str:
    base = rgb_filename
    for ext in [".JPG", ".JPEG", ".PNG", ".jpg", ".jpeg", ".png"]:
        if base.endswith(ext):
            base = base[:-len(ext)]
            break
    if base.endswith("_V"):
        return base[:-2]
    return base

# DJI_YYYYMMDDHHMMSS_#### pattern
_pat = re.compile(r"DJI_(\d{14})_(\d{4})", re.IGNORECASE)

def parse_ts_seq_from_name(name: str):
    """
    Return (datetime|None, int|None) from filename.
    """
    m = _pat.search(name)
    if not m:
        return None, None
    ts_str = m.group(1)
    seq_str = m.group(2)
    dt = datetime.strptime(ts_str, "%Y%m%d%H%M%S")
    return dt, int(seq_str)

def fmt_ts(dt: datetime | None):
    if dt is None:
        return "-"
    return dt.strftime("%H:%M:%S")

def human_ts_note(rgb_dt, th_dt):
    """
    Return human-friendly timestamp note, e.g.
    'Sesuai (09:57:08)' or 'Beda 1 dtk (RGB 09:57:08 vs TH 09:57:09)'
    """
    if rgb_dt is None or th_dt is None:
        return "Tidak terbaca"
    diff = int(abs((th_dt - rgb_dt).total_seconds()))
    if diff == 0:
        return f"Sesuai ({fmt_ts(rgb_dt)})"
    return f"Beda {diff} dtk (RGB {fmt_ts(rgb_dt)} vs TH {fmt_ts(th_dt)})"

# =========================
# INDEX THERMAL BY (seq) AND BY TIMESTAMP
# =========================
thermal_meta = []
for th_p in th_paths:
    th_dt, th_seq = parse_ts_seq_from_name(th_p.name)
    thermal_meta.append({
        "th_name": th_p.name,
        "th_path": str(th_p),
        "th_dt": th_dt,
        "th_seq": th_seq
    })

th_df = pd.DataFrame(thermal_meta)
th_df_ts = th_df.dropna(subset=["th_dt"]).sort_values("th_dt").reset_index(drop=True)

# quick map: seq -> list of thermal rows (in case duplicates)
th_by_seq = {}
for _, r in th_df.dropna(subset=["th_seq"]).iterrows():
    th_by_seq.setdefault(int(r["th_seq"]), []).append(r)

def find_nearest_thermal_same_seq(rgb_seq: int, rgb_dt: datetime):
    """
    Find thermal with same sequence number, closest by timestamp.
    """
    candidates = th_by_seq.get(rgb_seq, [])
    if not candidates:
        return None
    best = None
    best_diff = None
    for r in candidates:
        if r["th_dt"] is None or rgb_dt is None:
            continue
        diff = abs((r["th_dt"] - rgb_dt).total_seconds())
        if best is None or diff < best_diff:
            best = r
            best_diff = diff
    return best

# =========================
# BUILD TWO SIMPLE TABLES
# =========================
rows_ok = []
rows_no = []

# Tolerance to call it "offset kecil" (detik)
OFFSET_TOL_SEC = 2

for rgb_p in rgb_paths:
    frame_id = parse_frame_id_from_rgb_name(rgb_p.name)
    rgb_dt, rgb_seq = parse_ts_seq_from_name(rgb_p.name)

    expected_th_name = f"{frame_id}_T.npy"
    expected_th_path = RAW_TH_DIR / expected_th_name

    # CASE 1: Exact filename exists => Pairing OK
    if expected_th_path.exists():
        th_dt, _ = parse_ts_seq_from_name(expected_th_name)
        rows_ok.append({
            "RGB_File": rgb_p.name,
            "Thermal_File": expected_th_name,
            "Timestamp": human_ts_note(rgb_dt, th_dt),
            "Pairing": "OK"
        })
        continue

    # CASE 2: Exact missing => try find "thermal terdekat" dengan frame number sama (seq sama)
    near = None
    if rgb_seq is not None and rgb_dt is not None:
        near = find_nearest_thermal_same_seq(int(rgb_seq), rgb_dt)

    if near is not None and near["th_dt"] is not None and rgb_dt is not None:
        diff = abs((near["th_dt"] - rgb_dt).total_seconds())
        # kalau offset kecil -> "Tidak OK" (karena tidak sinkron nama/timestamp persis)
        if diff <= OFFSET_TOL_SEC:
            rows_no.append({
                "RGB_File": rgb_p.name,
                "Thermal_File_Terdekat": near["th_name"],
                "Timestamp": human_ts_note(rgb_dt, near["th_dt"]),
                "Pairing": "Tidak OK (offset waktu)"
            })
        else:
            # kasus lain: kemungkinan thermal benar-benar beda sesi / hilang
            rows_no.append({
                "RGB_File": rgb_p.name,
                "Thermal_File_Terdekat": near["th_name"],
                "Timestamp": human_ts_note(rgb_dt, near["th_dt"]),
                "Pairing": "Tidak OK (tidak sinkron)"
            })
    else:
        # tidak menemukan thermal dengan frame yang sama
        rows_no.append({
            "RGB_File": rgb_p.name,
            "Thermal_File_Terdekat": "-",
            "Timestamp": "Thermal tidak ditemukan",
            "Pairing": "Tidak OK (thermal tidak ada)"
        })

df_ok = pd.DataFrame(rows_ok)
df_no = pd.DataFrame(rows_no)

print("\n=== RINGKASAN ===")
print("Pairing OK       :", len(df_ok))
print("Pairing Tidak OK :", len(df_no))

display(df_ok.head(5))
display(df_no.head(5))

# =========================
# OPTIONAL: SAVE (kalau perlu)
# =========================
OUT_DIR = PROJECT_ROOT / "dokumentasi" / "pairing_audit"
OUT_DIR.mkdir(parents=True, exist_ok=True)

ok_csv = OUT_DIR / "table_pairing_ok.csv"
no_csv = OUT_DIR / "table_pairing_tidak_ok.csv"

df_ok.to_csv(ok_csv, index=False)
df_no.to_csv(no_csv, index=False)

print("\nSaved:")
print(ok_csv)
print(no_csv)


In [ ]:
print("\n=== RINGKASAN ===")
print("Pairing OK       :", len(df_ok))
print("Pairing Tidak OK :", len(df_no))

# =========================
# DISPLAY: 5 AWAL & 5 AKHIR
# =========================

print("\n--- TABLE 1: Pairing OK (5 Awal) ---")
display(df_ok.head(5))

print("\n--- TABLE 1: Pairing OK (5 Akhir) ---")
display(df_ok.tail(5))

print("\n--- TABLE 2: Pairing Tidak OK (5 Awal) ---")
display(df_no.head(5))

print("\n--- TABLE 2: Pairing Tidak OK (5 Akhir) ---")
display(df_no.tail(5))
